Contains code for **Game theory** and **Cooperation and coordination** experiments.  
**Game theory:**  
- Second price auction
- Beauty contest
- One-shot prisoner’s dilemma
- Finitely repeated prisoner’s dilemma
- One-shot public goods game
- Finitely repeated public goods game

**Cooperation and coordination:**  
- Infinitely repeated prisoner’s dilemma
- Battle of the sexes
- Stag hunt
- Minimum effort

In [1]:
import openai
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import random
import re
import time
import pickle
import copy
import json
import requests
filepath=r'./records/'
assert os.path.exists(filepath)

# Step 1: choose a LLM

## OSS LLMs: (except llama)
openchat-13b, wizardlm-13b, vicuna-13b, vicuna-7b, oasst-12b, chatglm2-6b

1. Install [fastchat](https://github.com/lm-sys/FastChat) and download the OSS LLM models from their official websites.  
2. Use fastchat to start an api server (see [here](https://github.com/lm-sys/FastChat/blob/main/docs/openai_api.md)).  
3. Then you can use openai api to call the LLMs.


In [ ]:
################## choose model here

model = "vicuna-13b-v1.5-16k"
# model = 'openchat_3.5'
# model = 'WizardLM-13B-V1.2'
# model = "vicuna-7b-v1.5-16k"
# model = 'oasst-sft-4-pythia-12b-epoch-3.5'
# model = 'chatglm2-6b'

In [36]:
openai.api_key = "EMPTY"
openai.api_base = "http://localhost:8000/v1"

def get_completion(prompt, model,temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

prompt = f"""Hello!"""
response = get_completion(prompt,model=model)
print(response)

class ChatApp:
    def __init__(self, system_message, init_message=None):
        self.messages = [
            {"role": "system", "content": system_message},
        ]

    def chat(self, message, model,temperature=0):
        self.messages.append({"role": "user", "content": message})
        response = openai.ChatCompletion.create(
                    model=model,
                    messages=self.messages,
                    temperature=temperature,
                )            
        self.messages.append({"role": "assistant", "content": response["choices"][0]["message"].content})
        return response["choices"][0]["message"]['content']
    
    def chat_wo_update(self, message, model, temperature=0):
        tmp_message=copy.deepcopy(self.messages)
        tmp_message.append({"role": "user", "content": message})
        response = openai.ChatCompletion.create(
                    model=model,
                    messages=tmp_message,
                    temperature=temperature,
                )

        return response["choices"][0]["message"]['content']

system_message="You are Bob, a friendly assistant."
Alice=ChatApp(system_message=system_message)
print(Alice.chat_wo_update(message="Hello, my name is Alice. What's your name?",model=model))
print(Alice.chat(message='Do you know my name?',model=model))

Hello! How can I help you today?
Hello Alice, my name is Bob. Nice to meet you!
I'm sorry, but as an AI language model, I don't have access to personal information unless you provide it to me. If you'd like to tell me your name, I'd be happy to address you by name.


## OSS LLMs: (llama)
llama2-13b, llama2-7b

1. Install [llama2-flask-api](https://github.com/unconv/llama2-flask-api) and download the Llama models from their official websites.  
2. Start an api server (see [here](https://github.com/unconv/llama2-flask-api)).  
3. Then you can use the following functions to call the LLMs.

In [39]:
def get_completion(prompt):
    messages = [{"role": "user", "content": prompt}]
#     messages = [{"role": "system", "content": 'You are Alice'},{"role": "user", "content": prompt}]
    response = requests.post("http://localhost:5000/chat", json={"messages":messages})
    response = json.loads(response.text)
    #print(res)
    return response['choices'][0]['message']["content"]

prompt = f"""Hello!"""
response = get_completion(prompt)
print(response)

class ChatApp:
    def __init__(self, system_message, init_message=None):
        self.messages = [
            {"role": "system", "content": system_message},
        ]

    def chat(self, message, model,temperature=0):
        self.messages.append({"role": "user", "content": message})
        response = requests.post("http://localhost:5000/chat", json={"messages":self.messages})
        response = json.loads(response.text)          
        self.messages.append({"role": "assistant", "content": response["choices"][0]["message"]['content']})
        return response["choices"][0]["message"]['content']
    
    def chat_wo_update(self, message, model, temperature=0):
        tmp_message=copy.deepcopy(self.messages)
        tmp_message.append({"role": "user", "content": message})
        response = requests.post("http://localhost:5000/chat", json={"messages":tmp_message})
        response = json.loads(response.text)
        return response["choices"][0]["message"]['content']


 Hello! I'm here to assist you with any questions or tasks you may have. Is there something specific you'd like to know or discuss? I'm here to help and provide information to the best of my ability. Please feel free to ask me anything!


## API LLMs - chat models
gpt-4, gpt-3.5

In [ ]:
############# set openai key here
key=''
openai.api_key = key

################## choose model here
model="gpt-3.5-turbo-1106"
# model='gpt-4-1106-preview'

In [ ]:
def get_completion(prompt, model,temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message["content"]

prompt = f"""Hello!"""
response = get_completion(prompt,model='gpt-3.5-turbo')
print(response)

class ChatApp:
    def __init__(self, system_message, init_message=None):
        self.messages = [
            {"role": "system", "content": system_message},
        ]

    def chat(self, message, model,temperature=0):
        self.messages.append({"role": "user", "content": message})
        response = openai.ChatCompletion.create(
                    model=model,
                    messages=self.messages,
                    temperature=temperature,
                )            
        self.messages.append({"role": "assistant", "content": response["choices"][0]["message"].content})
        return response["choices"][0]["message"]['content']
    
    def chat_wo_update(self, message, model, temperature=0):
        tmp_message=copy.deepcopy(self.messages)
        tmp_message.append({"role": "user", "content": message})
        response = openai.ChatCompletion.create(
                    model=model,
                    messages=tmp_message,
                    temperature=temperature,
                )

        return response["choices"][0]["message"]['content']


## API LLMs - completion models
bard, text-bison-001, text-davinci-003, text-davinci-002, claude-instant

In [ ]:
################## choose model here

# model='bard'
# model='text-bison-001'
# model="text-davinci-003"
model="text-davinci-002"
# model='Claude-Instant-Temp0'


############# set openai key here
key=''
openai.api_key = key

############# set bard token here
BARD_TOKEN=''

############# set palm token here (for text-bison-001)
palm_token = ''

############# set claude api key here 
claude_api_key = ""

In [ ]:
#################api function##################
import os
import openai
from bardapi import Bard
import google.generativeai as palm

original_proxy = os.environ.get('HTTP_PROXY', None)

print('Models available: gpt-3.5-turbo, gpt-4, \
text-davinci-003, text-davinci-002, text-bison-001, bard,\
Claude-instant, Claude-2-100k')
# print(palm_models)


import asyncio
from fastapi_poe.types import ProtocolMessage
from fastapi_poe.client import get_bot_response

async def get_claude_responses(api_key, model, message_prompt):
    complete_response = ''
    message = ProtocolMessage(role="user", content=message_prompt)
    async for partial in get_bot_response(messages=[message], 
                                        bot_name=model, # Claude-instant, Claude-2-100k, GPT-3.5-Turbo, Google-PaLM
                                        api_key=api_key):

        complete_response+=partial.text
#     print(complete_response)
    return complete_response

def convert_to_list(input_data):
    # if isinstance(input_data, str):
    #     return [input_data]
    # else:
    mlist = [item.strip() for item in input_data.split(",")]
    return mlist
# def get_completion(model, prompt):
async def get_completion(model, prompt):
    role_prompt = "You are a helpful assistant." # for gpt's only
    # message_prompt = input("Enter your message prompt: ")
    # models = input("Which models do you want to test: ")
    message_prompt = prompt
    models = model
    # models = [models]

    models = convert_to_list(models)
#     print(models)
    tasks = []
    for model in models:
#         print(model)
        if model == 'gpt-3.5-turbo' or model == 'gpt-4': 
            try: 
                response = openai.chat.completions.create(
                    model=model,
                    messages=[
                        {"role": "system", "content": role_prompt},
                        {"role": "user", "content": message_prompt}
                ]
                )
#                 print(response.choices[0].message.content)
                out=response.choices[0].message.content

            except Exception as e:
                print(f"{model} API failed with error: {str(e)}. Please wait a while and try again.")
                continue

        elif model == 'text-davinci-003' or model == 'text-davinci-002':
            try:
                response = openai.Completion.create(
                    model=model,
                    prompt = message_prompt,
                    temperature=0,
                    max_tokens=1000
                )
#                 print(response.choices[0].text.strip())
                out=response.choices[0].text.strip()

            except Exception as e:
                print(f"{model} API failed with error: {str(e)}. Please wait a while and try again.")
                continue

        elif model == 'text-bison-001': 
            os.environ['HTTP_PROXY'] = 'http://127.0.0.1:33210'
                # Enable the proxy
            try:
                palm.configure(api_key=palm_token)
                completion = palm.generate_text(
                    model='models/text-bison-001',
                    prompt=message_prompt,
                    temperature=0,
                    max_output_tokens=1000,
                )
#                 print(completion.result)
                out=completion.result
            except Exception as e:
                print(f"{model} API failed with error: {str(e)}.")
                continue
            finally:
                # Restore the original value of HTTP_PROXY
                os.environ.clear()  # Clear all environment variables
                os.environ.update(os.environ.copy())


        elif model == 'bard':
            try:
                # bard_token = input("Enter bard token (__Secure-1PSID): ")
                bard_token = BARD_TOKEN
                bard = Bard(token=bard_token)
                out=bard.get_answer(message_prompt)['content']
#                 print(out)
            except Exception as e:
                print(f"Bard API failed with error: {str(e)}. Please wait a while and try again.")
                continue


        elif model == 'Claude-instant' or model == 'Claude-2-100k' or model == 'ValidatorClaude' or model=='Claude-Instant-Temp0':
            try:
                api_key = claude_api_key
                out=get_claude_responses(api_key, model, message_prompt)
                tasks.append(out)
                out=await asyncio.gather(*tasks)
                out=out[0]
                return out
            except Exception as e:
                print(f"{model} API failed with error: {str(e)}.")
                continue
        
    return out



In [ ]:
import nest_asyncio
nest_asyncio.apply()


class ChatApp:
    def __init__(self, system_message, init_message=None):
        self.messages = [
            {"role": "system", "content": system_message},
        ]

    def chat(self, message, model,temperature=0):
        self.messages.append({"role": "user", "content": message})
        if len(self.messages)<=2:
            prompt='USER: '+self.messages[0]['content']+' '+self.messages[1]['content']+' AGENT: '
        else:
            prompt='USER: '+self.messages[0]['content']+' '+self.messages[1]['content']
            for m in self.messages[2:]:
                if m['role']=='user':
                    prompt+='USER: '+m['content']
                elif m['role']=='assistant':
                    prompt+='AGENT: '+m['content']
            prompt+=' AGENT: '
        response=asyncio.run(get_completion(model, prompt))
        self.messages.append({"role": "assistant", "content": response})
        return response
    
    def chat_wo_update(self, message, model, temperature=0):
        tmp_message=copy.deepcopy(self.messages)
        tmp_message.append({"role": "user", "content": message})
        if len(tmp_message)<=2:
            prompt='USER: '+tmp_message[0]['content']+' '+tmp_message[1]['content']+' AGENT: '
        else:
            prompt='USER: '+tmp_message[0]['content']+' '+tmp_message[1]['content']
            for m in tmp_message[2:]:
                if m['role']=='user':
                    prompt+='USER: '+m['content']
                elif m['role']=='assistant':
                    prompt+='AGENT: '+m['content']
            prompt+=' AGENT: '
        response=asyncio.run(get_completion(model, prompt))
        return response

# system_message="You are Bob, a friendly assistant."
# Alice=ChatApp(system_message=system_message)
# print(Alice.chat_wo_update(message="Hello, my name is Alice. What's your name?",model=model))
# print(Alice.chat(message='Do you know my name?',model=model))

# Step 2: choose a game and run experiments

## 1. Second price auction

In [16]:
def correct_json(s):
    prompt=f"""
The following string delimited by triple backticks is in json format, but there are some mistakes, and I cannot directly convert it to json by json.loads(). For example, there may be missing comma or quotes. Please help me correct the mistakes, and output the string in valid json format. Please only output the corrected string, and do not output any other things.
```
{s}
```
    """.strip()
    response = get_completion(prompt)
    return response

def parse_response(response): # return a json
    response = response.replace('Action','action')
    response = response.replace('Thought','thought')
    try:
        out=json.loads(response,strict=0)
    except:
        start_index = response.find('{')  
        end_index = response.rfind('}')  
        if end_index==-1:
            response=response.strip()+'}'
            
        start_index = response.find('{') 
        end_index = response.find('}') 
        if start_index != -1 and end_index != -1:
            extracted_content = response[start_index:end_index + 1]
            try:
                out=json.loads(extracted_content,strict=0)
            except:
                print('json error: ',extracted_content)
                corrected_json=correct_json(extracted_content)
                print('corrected:',corrected_json)
                start_index = corrected_json.find('{')  
                end_index = corrected_json.rfind('}')  
                extracted_content = corrected_json[start_index:end_index + 1]
                out=json.loads(extracted_content,strict=0)
        else:
            print('No json found in:',response)
            assert 0
    return out

def get_action(bot,model,display=False):
    message=f"""
Now let's start the auction. Tell me how you think and the bid you would like to place. Please answer in json format with keys 'thought' and 'bid'. For example, {{"thought": "xxx","bid": x}}. 
""".strip()
#     The 'bid' should be a number.
    res=bot.chat_wo_update(message, model=model)
    time.sleep(1)
    if display:
        print('-------get action---------')
        print(message)
        print(res)
    out=parse_response(res)
    action=out['bid']
    
    reason=out['thought']
    try:
        action=float(action)
    except:
        print('Invalid action:',out)
        assert 0
    return action,reason


In [41]:
nsess=10

expname='llama13b'

max_try=50
display_result=0

system_message_0 = """
Please act as a human bidder in an auction. You are participating in an auction with another bidder. There is only one item, and your private value of the item is {} points. You do not know the private value of the other bidder. 
You and the other bidder will simultaneously place a bid (can be 0 or any positive number). The bidder who places the higher bid will get the item (the other will not), and only need to pay a number of points equaling to the second-highest bid among all the bids.
If you get the item, your payoff is calculated as: payoff = your value of the item – the number of points you pay
If you do not get the item, your payoff is zero.
Your goal is to maximize your own payoff. You should first think carefully about what to do, and then make a decision.
""".strip()

for value in range(10,101,10):
    system_message=system_message_0.format(value)
    
    for sess in range(1,1+nsess):
        print(f'==============value={value},sess{sess}==============')

        recordfile=os.path.join(filepath,f'records_{expname}_value{value}_sess{sess}.pkl')
        if os.path.exists(recordfile):
            with open(recordfile, 'rb') as f:
                records = pickle.load(f)
            print('existed!')
            continue

        starttime=time.time()
        for _ in range(max_try):
            try:
                bot=ChatApp(system_message=system_message)
                action,reason=get_action(bot,model=model,display=display_result)
                records=[{'action':action,'reason':reason}]
                print('time:',time.time()-starttime)
                # save file
                recordfile=os.path.join(filepath,f'records_{expname}_value{value}_sess{sess}.pkl')
                with open(recordfile, 'wb') as f:
                    pickle.dump(records, f)
                break
            except Exception as e:
                print(e)
                print('error!sleep 30s...')
                time.sleep(30)
       

==============value=10,sess1==============
time: 4.838624715805054
==============value=10,sess2==============
time: 4.813868999481201
==============value=10,sess3==============
time: 4.794914484024048
==============value=10,sess4==============
time: 4.819051504135132
==============value=10,sess5==============
time: 4.93616795539856
==============value=10,sess6==============
time: 4.779558420181274
==============value=10,sess7==============
time: 4.826843023300171
==============value=10,sess8==============
time: 4.820796012878418
==============value=10,sess9==============
time: 4.754726409912109
==============value=10,sess10==============
time: 4.845721006393433
==============value=20,sess1==============
time: 5.542486667633057
==============value=20,sess2==============
time: 5.472866535186768
==============value=20,sess3==============
time: 5.48654580116272
==============value=20,sess4==============
time: 5.57664680480957
==============value=20,sess5==============
time: 5.4849414825439

corrected: {
"thought": "Since my private value of the item is 90 points, I should bid a number that is higher than 90 - 10 = 80 points, to ensure that I have a chance to win the item. However, I don't want to bid too high and end up overpaying for the item. Let's see... I'll bid 85 points.",
"bid": 85
}
time: 9.223864316940308
==============value=90,sess6==============
json error:  {
"thought": "Since my private value of the item is 90 points, I should bid a number that is higher than 90 - 10 = 80 points, to ensure that I have a chance to win the item. However, I don't want to bid too high and end up overpaying for the item. Let's see... I'll bid 85 points."
"bid": 85
}
corrected: {
"thought": "Since my private value of the item is 90 points, I should bid a number that is higher than 90 - 10 = 80 points, to ensure that I have a chance to win the item. However, I don't want to bid too high and end up overpaying for the item. Let's see... I'll bid 85 points.",
"bid": 85
}
time: 8.653895

In [42]:
# calculate payoffdeviation from NE
nsess=10
nround=10
round=nround

result=[]
for value in range(10,101,10):
    for sess in range(1,nsess+1):
        recordfile=os.path.join(filepath,f'records_{expname}_value{value}_sess{sess}.pkl')
        with open(recordfile, 'rb') as f:
            records= pickle.load(f)
        for record in records:
            tmp=record['action']
#             print(tmp)
            result.append(abs(tmp-value)/value)
        
print(len(result),np.mean(result))
dev=np.mean(result)
dev

100 0.21692460317460316


0.21692460317460316

## 2. Beauty contest

In [5]:
max_number=30

system_message = f"""
Please act as a human player. You are playing a game with another human player (called the opponent) for only one round. You and the opponent will simultaneously choose a number between 0 and {max_number} (including 0 and {max_number}). The winner is the player whose number is closest to two thirds of the average of all chosen numbers. The winner will earn 100 points, and the other player will earn nothing. If there is a tie, both players will earn 50 points.
Your goal is to maximize your own earnings. You should first think carefully about what to do, and then make a decision.
""".strip()

print(system_message)

Please act as a human player. You are playing a game with another human player (called the opponent) for only one round. You and the opponent will simultaneously choose a number between 0 and 30 (including 0 and 30). The winner is the player whose number is closest to two thirds of the average of all chosen numbers. The winner will earn 100 points, and the other player will earn nothing. If there is a tie, both players will earn 50 points.
Your goal is to maximize your own earnings. You should first think carefully about what to do, and then make a decision.


In [6]:
def correct_json(s):
    prompt=f"""
The following string delimited by triple backticks is in json format, but there are some mistakes, and I cannot directly convert it to json by json.loads(). For example, there may be missing comma or quotes. Please help me correct the mistakes, and output the string in valid json format. Please only output the corrected string, and do not output any other things.
```
{s}
```
    """.strip()
    response = get_completion(prompt,model=model)
    return response
s="""{
  "thought": "aaa"
  "action": J
}"""
# print(correct_json(s))


summarize_action=f"""Given the following paragraph delimited by triple backticks:
```
<out>
```
Please summarize how he thought, and the number he chose in the first person from above paragraph in json format with keys 'thought' and 'number'. The 'number' should be between 0 and 100 (including 0 and 100).
""".strip()

In [7]:
def parse_response(response): # return a json
    response = response.replace('Action','action')
    response = response.replace('Thought','thought')
    try:
        out=json.loads(response,strict=0)
    except:
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if end_index==-1:
            response=response.strip()+'}'
            
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if start_index != -1 and end_index != -1:
            extracted_content = response[start_index:end_index + 1]
            try:
                out=json.loads(extracted_content,strict=0)
            except:
                print('json error: ',extracted_content)
                corrected_json=correct_json(extracted_content)
                print('corrected:',corrected_json)
                start_index = corrected_json.find('{')  # 查找第一个'{'的索引
                end_index = corrected_json.rfind('}')  # 查找最后一个'}'的索引
                extracted_content = corrected_json[start_index:end_index + 1]
                out=json.loads(extracted_content,strict=0)
        else:
            prompt=summarize_action.replace('<out>',response)
            out=gpt_completion(prompt)
            out=json.loads(out)
#             print('No json found in:',response)
#             assert 0
    return out


def get_action(bot,model,display=False):
    message=f"""
Now let's start the game. Tell me how you think and the number you would like to choose. Please answer in json format with keys 'thought' and 'number'. The 'number' should be between 0 and {max_number} (including 0 and {max_number}).
""".strip()
#     message=f"""
# Now let's start the game. Tell me how you think and the number you would like to choose.
# """.strip()
    res=bot.chat_wo_update(message, model=model)
    time.sleep(1)
    if display:
        print('-------get action---------')
        print(message)
        print(res)
    out=parse_response(res)
    action=out['number']
    
    reason=out['thought']
    try:
        action=float(action)
    except:
        print('Invalid action:',out)
        assert 0
    return action,reason


In [ ]:
nsess=10

expname='llama13b'

max_try=50
display_result=0

system_message_0 = """
Please act as a human player. You are playing a game with another human player (called the opponent) for only one round. You and the opponent will simultaneously choose a number between 0 and {} (including 0 and {}). The winner is the player whose number is closest to two thirds of the average of all chosen numbers. The winner will earn 100 points, and the other player will earn nothing. If there is a tie, both players will earn 50 points.
Your goal is to maximize your own earnings. You should first think carefully about what to do, and then make a decision.
""".strip()
for max_number in range(10,101,10):
    system_message=system_message_0.format(max_number,max_number)
#     print(system_message)
    
    for sess in range(1,1+nsess):
        print(f'==============max_number={max_number},sess{sess}==============')

        recordfile=os.path.join(filepath,f'records_{expname}_max{max_number}_sess{sess}.pkl')
        if os.path.exists(recordfile):
            with open(recordfile, 'rb') as f:
                records = pickle.load(f)
            print('existed!')
            continue

        starttime=time.time()
        for _ in range(max_try):
            try:
                bot=ChatApp(system_message=system_message)
                action,reason=get_action(bot,model=model,display=display_result)
                records=[{'action':action,'reason':reason}]
                print('time:',time.time()-starttime)
                # save file
                recordfile=os.path.join(filepath,f'records_{expname}_max{max_number}_sess{sess}.pkl')
                with open(recordfile, 'wb') as f:
                    pickle.dump(records, f)
                break
            except Exception as e:
                print(e)
                print('error!sleep 30s...')
                time.sleep(30)   
       

In [ ]:
# payoffdeviation from NE
nsess=10
nround=10
round=nround


result=[]
for max_number in range(10,101,10):
    for sess in range(1,nsess+1):
        recordfile=os.path.join(filepath,f'records_{expname}_max{max_number}_sess{sess}.pkl')
        with open(recordfile, 'rb') as f:
            records= pickle.load(f)
        for record in records:
            tmp=record['action']
#             print(tmp)
            result.append(tmp/max_number)
        
print(len(result),np.mean(result))
dev=np.mean(result)
dev

## 3. One-shot prisoner’s dilemma

In [46]:
system_message = """
Please act as a human player. You are playing a game with another human player (called the opponent) for only one round. You and the opponent will simultaneously choose an action between F and J.
{payoff_str}
Your goal is to maximize your own earnings. You should first think carefully about what to do, and then choose one of the two actions: F or J.
""".strip()


aa=f'If you choose J and the opponent chooses J, you earn 40 points and the opponent earns 40 points.'
ab=f'If you choose J and the opponent chooses F, you earn 12 points and the opponent earns 50 points.'
ba=f'If you choose F and the opponent chooses J, you earn 50 points and the opponent earns 12 points.'
bb=f'If you choose F and the opponent chooses F, you earn 25 points and the opponent earns 25 points.'
payoff=[aa,ab,ba,bb]
payoff_str=payoff[0]+'\n'+payoff[1]+'\n'+payoff[2]+'\n'+payoff[3]

tmp_system_message=system_message.format(payoff_str=payoff_str)
print(tmp_system_message)

Please act as a human player. You are playing a game with another human player (called the opponent) for only one round. You and the opponent will simultaneously choose an action between F and J.
If you choose J and the opponent chooses J, you earn 40 points and the opponent earns 40 points.
If you choose J and the opponent chooses F, you earn 12 points and the opponent earns 50 points.
If you choose F and the opponent chooses J, you earn 50 points and the opponent earns 12 points.
If you choose F and the opponent chooses F, you earn 25 points and the opponent earns 25 points.
Your goal is to maximize your own earnings. You should first think carefully about what to do, and then choose one of the two actions: F or J.


In [47]:
def correct_json(s):
    tmp=s.split('"')
    newlist=[]
    for x in tmp:
        striped=x.strip()
        if striped=='': # 缺少逗号
            newlist.append(',')
        elif striped[0]==':' and striped!=':':
            newlist.extend([':',striped[1:-1].strip(),'}'])
        else:
            newlist.append(striped)
    newstr='"'.join(newlist)
    try:
        a=json.loads(newstr)
    except:
        prompt=f"""
The following string delimited by triple backticks is in json format, but there are some mistakes, and I cannot directly convert it to json by json.loads(). For example, there may be missing comma or quotes. Please help me correct the mistakes, and output the string in valid json format. Please only output the corrected string, and do not output any other things.
```
{s}
```
        """.strip()
        response = gpt_completion(prompt)
        return response
    return newstr

s="""{
  "thought": "aaa"
  "action": J
}"""
# print(correct_json(s))

summarize_action=f"""Given the following paragraph delimited by triple backticks:
```
<out>
```
Please summarize how he thought and the action he choose (F or J) in the first person from above paragraph in json format with keys 'thought' and 'action'. The 'action' should be F or J.
""".strip()

In [48]:
def parse_response(response): # return a json
    response = response.replace('Action','action')
    response = response.replace('Thought','thought')
    try:
        out=json.loads(response,strict=0)
    except:
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if end_index==-1:
            response=response.strip()+'}'
            
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if start_index != -1 and end_index != -1:
            extracted_content = response[start_index:end_index + 1]
            try:
                out=json.loads(extracted_content,strict=0)
            except:
                print('json error: ',extracted_content)
                corrected_json=correct_json(extracted_content)
                print('corrected:',corrected_json)
                start_index = corrected_json.find('{')  # 查找第一个'{'的索引
                end_index = corrected_json.rfind('}')  # 查找最后一个'}'的索引
                extracted_content = corrected_json[start_index:end_index + 1]
                out=json.loads(extracted_content,strict=0)
        else:
            print('No json found in:',response)
            prompt=summarize_action.replace('<out>',response)
            out=gpt_completion(prompt)
            print('Summarize json:',out)
            out=json.loads(out,strict=0)
    return out

def get_action(bot,model,display=False):
    message=f"""
Now let's start the game. Tell me how you think and the action you would like to choose. Please answer in json format with keys 'thought' and 'action'. For example, {{"thought": "xxx","action": x}}. The 'action' should be F or J. 
""".strip()
#     For example, {{"thought": "xxx","action": x}}. The 'action' should be F or J. 
#     message=f"""
# Now let's start the game. Tell me how you think and the action you would like to choose.
# """.strip()
    res=bot.chat_wo_update(message, model=model)
    time.sleep(1)
    if display:
        print('-------get action---------')
        print(message)
        print(res)
    out=parse_response(res)
    try:
        action=out['action']
        reason=out['thought']
    except:
        print('key error:',res)
        prompt=summarize_action.replace('<out>',res)
        out=gpt_completion(prompt)
        out=parse_response(out)
        print('corrected:',out)
        action=out['action']
        reason=out['thought']
        
    try:
        assert action in ['F','J']
    except:
        print('Invalid action:',out)
        assert 0
    return action,reason


In [52]:
nsess=50

expname='llama13b'


max_try=50
display_result=0

for sess in range(1,1+nsess):
    print(f'==============sess{sess}==============')
    system_message = """
Please act as a human player. You are playing a game with another human player (called the opponent) for only one round. You and the opponent will simultaneously choose an action between F and J.
{payoff_str}
Your goal is to maximize your own earnings. You should first think carefully about what to do, and then choose one of the two actions: F or J.
""".strip()

    aa=f'If you choose J and the opponent chooses J, you earn 40 points and the opponent earns 40 points.'
    ab=f'If you choose J and the opponent chooses F, you earn 12 points and the opponent earns 50 points.'
    ba=f'If you choose F and the opponent chooses J, you earn 50 points and the opponent earns 12 points.'
    bb=f'If you choose F and the opponent chooses F, you earn 25 points and the opponent earns 25 points.'
    payoff=[aa,ab,ba,bb]
    
    ######################## init players and records
    N=2
    players=[]
    payoff=[aa,ab,ba,bb]
    for _ in range(N):
        random.shuffle(payoff)
        payoff_str=payoff[0]+'\n'+payoff[1]+'\n'+payoff[2]+'\n'+payoff[3]
        tmp_system_message=system_message.format(payoff_str=payoff_str)
    #     print(tmp_system_message)
        bot=ChatApp(system_message=tmp_system_message)
        players.append(bot)
    records=[]
    for _ in range(N):    
        df = pd.DataFrame(columns=['Round','Your choice','Co-player choice','Earnings','Reason of choice'])
        records.append(df)

    ######################## start game
    # if this sess done, continue
    recordfile=os.path.join(filepath,f'records_{expname}_sess{sess}.pkl')
    playerfile=os.path.join(filepath,f'players_{expname}_sess{sess}.pkl')
    if os.path.exists(recordfile) and os.path.exists(playerfile):
        with open(recordfile, 'rb') as f:
            records = pickle.load(f)
        with open(playerfile, 'rb') as f:
            players = pickle.load(f)
        print('existed!')
        continue
    starttime=time.time()
    for _ in range(max_try):
        try:
            # play
            tmp_records=[]
            for i in tqdm(range(N)):
                bot=players[i]
                record=records[i]
                action,reason=get_action(bot,model=model,display=display_result)
                tmp_records.append([1,action,'NULL',0,reason])
                # ['Round','Your choice','Co-player choice','Earnings','Reason of choice']

            # calculate total public
            c1,c2=tmp_records[0][1],tmp_records[1][1]
            if c1=='J' and c2=='J':
                e1,e2=40,40
            elif c1=='J' and c2=='F':
                e1,e2=12,50
            elif c1=='F' and c2=='J':
                e1,e2=50,12
            elif c1=='F' and c2=='F':
                e1,e2=25,25
            else:
                print(c1,c2)
                assert False
            # ['Round','Your choice','Co-player choice','Earnings','Reason of choice']
            tmp_records[0][2]=c2
            tmp_records[1][2]=c1
            tmp_records[0][3]=e1
            tmp_records[1][3]=e2

            # update records for all
            for i in range(N):
                record=records[i]
                record.loc[len(record.index)]=tmp_records[i]
            print('time:',time.time()-starttime)

            # save file
            recordfile=os.path.join(filepath,f'records_{expname}_sess{sess}.pkl')
            playerfile=os.path.join(filepath,f'players_{expname}_sess{sess}.pkl')
            with open(recordfile, 'wb') as f:
                pickle.dump(records, f)
            with open(playerfile, 'wb') as f:
                pickle.dump(players, f)
            break
        except Exception as e:
            print(e)
            print('error!sleep 30s...')
            time.sleep(30)   

==============sess1==============


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.46s/it]


time: 4.922423362731934
==============sess2==============


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:07<00:00,  3.68s/it]


json error:  {
"thought": "I will choose the action that has the highest expected value. Let's see, if the opponent chooses F, I will earn 25 points and the opponent will earn 25 points, so the expected value of choosing F is 50 points. If the opponent chooses J, I will earn 40 points and the opponent will earn 40 points, so the expected value of choosing J is also 50 points. Since both options have the same expected value, I will choose the one that has the higher probability of being chosen by the opponent. Based on the game's rules, the opponent is more likely to choose F than J, so I will choose J."
"action": "J"}
corrected: {"thought":"I will choose the action that has the highest expected value. Let's see, if the opponent chooses F, I will earn 25 points and the opponent will earn 25 points, so the expected value of choosing F is 50 points. If the opponent chooses J, I will earn 40 points and the opponent will earn 40 points, so the expected value of choosing J is also 50 points.

...

time: 5.005434036254883
==============sess49==============


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.12s/it]


time: 6.256823539733887
==============sess50==============


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.20s/it]

time: 4.403552770614624


In [54]:
# NE choice rate (defect rate)

allaction=[]

for sess in range(1,1+nsess):    
    recordfile=os.path.join(filepath,f'records_{expname}_sess{sess}.pkl')
    with open(recordfile, 'rb') as f:
        records= pickle.load(f)
    for record in records:
        allaction.extend(record['Your choice'].tolist())


out=allaction.count('F')/len(allaction)
print(out)
# print('%.3f'%(out))

0.78


## 4. Finitely repeated prisoner’s dilemma

In [ ]:
nround=10

system_message = """
Please act as a human player. You are playing a game with another human player (called the opponent) for {nround} rounds.
In each round, you and the opponent will simultaneously choose an action between F and J.
{payoff_str}
Before you choose an action, your actions, the opponent’s actions and your earnings in each of the previous rounds will be shown. Your goal is to maximize your own total earnings in all {nround} rounds. In each round, you should first think carefully about what to do, and then choose one of the two actions: F or J.
""".strip()

aa=f'If you choose J and the opponent chooses J, you earn 40 points and the opponent earns 40 points in this round.'
ab=f'If you choose J and the opponent chooses F, you earn 12 points and the opponent earns 50 points in this round.'
ba=f'If you choose F and the opponent chooses J, you earn 50 points and the opponent earns 12 points in this round.'
bb=f'If you choose F and the opponent chooses F, you earn 25 points and the opponent earns 25 points in this round.'
payoff=[aa,ab,ba,bb]
payoff_str=payoff[0]+'\n'+payoff[1]+'\n'+payoff[2]+'\n'+payoff[3]

tmp_system_message=system_message.format(nround=nround, payoff_str=payoff_str)
print(tmp_system_message)

In [18]:
def correct_json(s):
    tmp=s.split('"')
    newlist=[]
    for x in tmp:
        striped=x.strip()
        if striped=='': # 缺少逗号
            newlist.append(',')
        elif striped[0]==':' and striped!=':':
            newlist.extend([':',striped[1:-1].strip(),'}'])
        else:
            newlist.append(striped)
    newstr='"'.join(newlist)
    try:
        a=json.loads(newstr)
    except:
        prompt=f"""
The following string delimited by triple backticks is in json format, but there are some mistakes, and I cannot directly convert it to json by json.loads(). For example, there may be missing comma or quotes. Please help me correct the mistakes, and output the string in valid json format. Please only output the corrected string, and do not output any other things.
```
{s}
```
        """.strip()
        response = gpt_completion(prompt)
        return response
    return newstr

s="""{
  "thought": "aaa"
  "action": J
}"""
# print(correct_json(s))

summarize_action=f"""Given the following paragraph delimited by triple backticks:
```
<out>
```
Please summarize how he thought and the action he choose (F or J) in the first person from above paragraph in json format with keys 'thought' and 'action'.
""".strip()

In [19]:
def get_history(record):
    history=''
    for i in record.index:
        round=i+1
        choice=record.loc[i,'Your choice']
        cochoice=record.loc[i,'Co-player choice']
        earning=record.loc[i,'Earnings']
        tmp=f"""In round-{round}, you chose {choice} and the opponent chose {cochoice}, you earn {earning} points."""
        history+=tmp
        history+='\n'
    return history
def parse_response(response): # return a json
    response = response.replace('Action','action')
    response = response.replace('Thought','thought')
    try:
        out=json.loads(response,strict=0)
    except:
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if end_index==-1:
            response=response.strip()+'}'
            
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if start_index != -1 and end_index != -1:
            extracted_content = response[start_index:end_index + 1]
            try:
                out=json.loads(extracted_content,strict=0)
            except:
                print('json error: ',extracted_content)
                corrected_json=correct_json(extracted_content)
                print('corrected:',corrected_json)
                start_index = corrected_json.find('{')  # 查找第一个'{'的索引
                end_index = corrected_json.rfind('}')  # 查找最后一个'}'的索引
                extracted_content = corrected_json[start_index:end_index + 1]
                out=json.loads(extracted_content,strict=0)
        else:
            print('No json found in:',response)
            assert 0
    return out

In [24]:
def get_action(bot,record,model,display=False):
    history=get_history(record)
    if len(record)==0:
        message=f"""
It is round-{round} out of {nround} rounds now. Tell me how you think and the action you would like to choose. Please answer in json format with keys 'thought' and 'action'. The 'action' should be F or J. 
""".strip()
#         For example, {{"thought": "xxx","action": x}}. The 'action' should be F or J. 
    else:
        message=f"""The history of the game is listed as follows delimited by triple backticks.
```
{history}
```
It is round-{round} out of {nround} rounds now. Tell me how you think and the action you would like to choose. Please answer in json format with keys 'thought' and 'action'. The 'action' should be F or J. 
""".strip()
    res=bot.chat_wo_update(message, model=model)
    time.sleep(3)
    if display:
        print('-------get action---------')
        print(message)
        print(res)
    out=parse_response(res)
    try:
        action=out['action']
        reason=out['thought']
    except:
        print('key error:',res)
        prompt=summarize_action.replace('<out>',res)
        out=gpt_completion(prompt)
        out=parse_response(out)
        print('corrected:',out)
        action=out['action']
        reason=out['thought']
#         assert 0
    try:
        assert action in ['F','J']
    except:
        print('Invalid action:',out)
        assert 0
    return action,reason


In [44]:
nround=10
nsess=10

expname='llama13b'


max_try=50
display_result=0


for sess in range(1,1+nsess):
    print(f'==============sess{sess}==============')
    system_message = """
Please act as a human player. You are playing a game with another human player (called the opponent) for {nround} rounds.
In each round, you and the opponent will simultaneously choose an action between F and J.
{payoff_str}
Before you choose an action, your actions, the opponent’s actions and your earnings in each of the previous rounds will be shown. Your goal is to maximize your own total earnings in all {nround} rounds. In each round, you should first think carefully about what to do, and then choose one of the two actions: F or J.
    """.strip()

    aa=f'If you choose J and the opponent chooses J, you earn 40 points and the opponent earns 40 points in this round.'
    ab=f'If you choose J and the opponent chooses F, you earn 12 points and the opponent earns 50 points in this round.'
    ba=f'If you choose F and the opponent chooses J, you earn 50 points and the opponent earns 12 points in this round.'
    bb=f'If you choose F and the opponent chooses F, you earn 25 points and the opponent earns 25 points in this round.'
    payoff=[aa,ab,ba,bb]
    
    ######################## init players and records
    N=2
    players=[]
    payoff=[aa,ab,ba,bb]
    for _ in tqdm(range(N)):
        random.shuffle(payoff)
        payoff_str=payoff[0]+'\n'+payoff[1]+'\n'+payoff[2]+'\n'+payoff[3]
        tmp_system_message=system_message.format(nround=nround, payoff_str=payoff_str)
    #     print(tmp_system_message)
        bot=ChatApp(system_message=tmp_system_message)
        players.append(bot)
    records=[]
    for _ in range(N):    
        df = pd.DataFrame(columns=['Round','Your choice','Co-player choice','Earnings','Reason of choice'])
        records.append(df)

    ######################## start game

    starttime=time.time()
    for round in range(1,nround+1):
        print('-----------------------')
        print(f'round-{round}')

        # if this round done, continue
        recordfile=os.path.join(filepath,f'records_finPD_{expname}_{nround}_sess{sess}_round{round}.pkl')
        playerfile=os.path.join(filepath,f'players_finPD_{expname}_{nround}_sess{sess}_round{round}.pkl')
        if os.path.exists(recordfile) and os.path.exists(playerfile):
            with open(recordfile, 'rb') as f:
                records = pickle.load(f)
            with open(playerfile, 'rb') as f:
                players = pickle.load(f)
            print('existed!')
            continue

#         for _ in range(max_try):
#             try:
        if round>1:
            prev_round=round-1
            recordfile=os.path.join(filepath,f'records_finPD_{expname}_{nround}_sess{sess}_round{prev_round}.pkl')
            playerfile=os.path.join(filepath,f'players_finPD_{expname}_{nround}_sess{sess}_round{prev_round}.pkl')
            with open(recordfile, 'rb') as f:
                records = pickle.load(f)
            with open(playerfile, 'rb') as f:
                players = pickle.load(f)
        # play
        tmp_records=[]
        for i in tqdm(range(N)):
            bot=players[i]
            record=records[i]
            action,reason=get_action(bot,record,model=model,display=display_result)
            tmp_records.append([round,action,'NULL',0,reason])
            # ['Round','Your choice','Co-player choice','Earnings','Reason of choice']

        # calculate total public
        c1,c2=tmp_records[0][1],tmp_records[1][1]
        if c1=='J' and c2=='J':
            e1,e2=40,40
        elif c1=='J' and c2=='F':
            e1,e2=12,50
        elif c1=='F' and c2=='J':
            e1,e2=50,12
        elif c1=='F' and c2=='F':
            e1,e2=25,25
        else:
            print(c1,c2)
            assert False
        # ['Round','Your choice','Co-player choice','Earnings','Reason of choice']
        tmp_records[0][2]=c2
        tmp_records[1][2]=c1
        tmp_records[0][3]=e1
        tmp_records[1][3]=e2

        # update records for all
        for i in range(N):
            record=records[i]
            record.loc[len(record.index)]=tmp_records[i]
        print('time:',time.time()-starttime)

        # save file
        recordfile=os.path.join(filepath,f'records_finPD_{expname}_{nround}_sess{sess}_round{round}.pkl')
        playerfile=os.path.join(filepath,f'players_finPD_{expname}_{nround}_sess{sess}_round{round}.pkl')
        with open(recordfile, 'wb') as f:
            pickle.dump(records, f)
        with open(playerfile, 'wb') as f:
            pickle.dump(players, f)
#                 break
#             except Exception as e:
#                 print(e)
#                 print('error!sleep 30s...')
#                 time.sleep(30)

==============sess1==============


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 6186.29it/s]


-----------------------
round-1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.90s/it]


time: 9.815633296966553
-----------------------
round-2


...

time: 93.66999530792236
-----------------------
round-9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.29s/it]


time: 106.25659942626953
-----------------------
round-10


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:12<00:00,  6.00s/it]

time: 118.27615332603455


In [46]:
# NE choice rate (defect rate)
nsess=10
nround=10
round=nround

allaction=[]

for sess in range(1,1+nsess):    
    recordfile=os.path.join(filepath,f'records_finPD_{expname}_{nround}_sess{sess}_round{round}.pkl')
    with open(recordfile, 'rb') as f:
        records= pickle.load(f)
    for record in records:
        allaction.extend(record['Your choice'].tolist())


out=allaction.count('F')/len(allaction)
print(out)
# print('%.3f'%(out))

0.315


## 5. One-shot public goods game

In [24]:
N,G,K,Z=4, 1.2, 0.3, 100 

assert abs(G-N*K)<0.000001

system_message = f"""
Please act as a human player. You and {N-1} other players are playing a game for only one round. There is a shared public account and each player has a private account. Each player will be given {Z} points. Then, all players simultaneously allocate these points into private account and public account. 
The points you allocate to your private account will be exchanged for earnings at a rate of 1:1, and these earnings will be received only by you. The total number of points in the public account equals to the sum of points allocated to the public account by all players (including yourself). These points will be exchanged for public earnings at a rate of 1:{G}, and these earnings will be equally shared among all {N} players, which means that each point in the public account will yield an earning of {K} points for each player. 
In sum, your earnings can be described as: Your earnings = (Points in your private account * 1) + (Total points in public account * {K})
Your goal is to maximize your own earnings. You should first think carefully about what to do, and then make a decision.
""".strip()

print(system_message)

Please act as a human player. You and 3 other players are playing a game for only one round. There is a shared public account and each player has a private account. Each player will be given 100 points. Then, all players simultaneously allocate these points into private account and public account. 
The points you allocate to your private account will be exchanged for earnings at a rate of 1:1, and these earnings will be received only by you. The total number of points in the public account equals to the sum of points allocated to the public account by all players (including yourself). These points will be exchanged for public earnings at a rate of 1:1.2, and these earnings will be equally shared among all 4 players, which means that each point in the public account will yield an earning of 0.3 points for each player. 
In sum, your earnings can be described as: Your earnings = (Points in your private account * 1) + (Total points in public account * 0.3)
Your goal is to maximize your own

In [25]:
summarize_action=f"""Given the following paragraph delimited by triple backticks:
```
<out>
```
Please summarize how he thought, and the number of points allocated in private/public account in the first person from above paragraph in json format with keys 'thought', 'private' and 'public'. The value of 'private' and 'public' should be a number.
""".strip()

In [27]:
def parse_response(response): # return a json
    response = response.replace('Action','action')
    response = response.replace('Thought','thought')
    try:
        out=json.loads(response,strict=0)
    except:
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if end_index==-1:
            response=response.strip()+'}'
            
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if start_index != -1 and end_index != -1:
            extracted_content = response[start_index:end_index + 1]
            try:
                out=json.loads(extracted_content,strict=0)
            except:
                print('json error: ',extracted_content)
                corrected_json=correct_json(extracted_content)
                print('corrected:',corrected_json)
                start_index = corrected_json.find('{')  # 查找第一个'{'的索引
                end_index = corrected_json.rfind('}')  # 查找最后一个'}'的索引
                extracted_content = corrected_json[start_index:end_index + 1]
                out=json.loads(extracted_content,strict=0)
        else:
            print('No json found in:',response)
            assert 0
    return out

def get_action(bot,model,display=False):
    message=f"""
Now let's start the game. You are given {Z} points. Tell me how you think and how much you would like to allocate to the private and public account, respectively. Please answer in json format with keys 'thought', 'private', and 'public'. The sum of private and public should equal {Z}. 
""".strip()
    res=bot.chat_wo_update(message, model=model)
    time.sleep(1)
    if display:
        print('-------get action---------')
        print(message)
        print(res)
    out=parse_response(res)
    try:
        private=out['private']
        public=out['public']
        reason=out['thought']
        private,public=int(private),int(public)
        
    except:
        print('key error:',res)
        prompt=summarize_action.replace('<out>',res)
        out=gpt_completion(prompt)
        out=parse_response(out)
        print('corrected:',out)
        private=out['private']
        public=out['public']
        reason=out['thought']
        
    try:
        assert abs(private+public-Z)<0.0001
    except:
        print('private+public!=Z',[private,public])
        assert False
    
    return private,public,reason


In [30]:
###########################
nsess=20

expname='llama13b'

max_try=50
display_result=0

system_message = f"""
Please act as a human player. You and {N-1} other players are playing a game for only one round. There is a shared public account and each player has a private account. Each player will be given {Z} points. Then, all players simultaneously allocate these points into private account and public account. 
The points you allocate to your private account will be exchanged for earnings at a rate of 1:1, and these earnings will be received only by you. The total number of points in the public account equals to the sum of points allocated to the public account by all players (including yourself). These points will be exchanged for public earnings at a rate of 1:{G}, and these earnings will be equally shared among all {N} players, which means that each point in the public account will yield an earning of {K} points for each player. 
In sum, your earnings can be described as: Your earnings = (Points in your private account * 1) + (Total points in public account * {K})
Your goal is to maximize your own earnings. You should first think carefully about what to do, and then make a decision.
""".strip()

for sess in range(1,1+nsess):
    print(f'==============sess{sess}==============')
    
    ######################## init players and records
    players=[]
    for _ in tqdm(range(N)):
        bot=ChatApp(system_message=system_message)
        players.append(bot)
    records=[]
    cols=['Period','Private','Public','Total public','Earnings','Reason']
    for _ in range(N):    
        df = pd.DataFrame(columns=cols)
        records.append(df)

    ######################## start game
    # if this sess done, continue
    recordfile=os.path.join(filepath,f'records_{expname}_{N}_{G}_{K}_{Z}_sess{sess}.pkl')
    playerfile=os.path.join(filepath,f'players_{expname}_{N}_{G}_{K}_{Z}_sess{sess}.pkl')
    if os.path.exists(recordfile) and os.path.exists(playerfile):
        with open(recordfile, 'rb') as f:
            records = pickle.load(f)
        with open(playerfile, 'rb') as f:
            players = pickle.load(f)
        print('existed!')
        continue
    starttime=time.time()
    for _ in range(max_try):
        try:
            # play
            tmp_records=[]
            for i in tqdm(range(N)):
                bot=players[i]
                record=records[i]
                private,public,reason=get_action(bot,model=model,display=display_result)
                tmp_records.append([1,private,public,0,0,reason])
                # ['Period','Private','Public','Total public','Earnings','Reason']

            # calculate total public
            total_public=sum([x[2] for x in tmp_records])

            # calculate earnings for all
            for r in tmp_records:
                r[3]=total_public
                r[4]=r[1]+total_public*K

            # update records for all
            for i in range(N):
                record=records[i]
                record.loc[len(record.index)]=tmp_records[i]

            print('time:',time.time()-starttime)


            # save file
            recordfile=os.path.join(filepath,f'records_{expname}_{N}_{G}_{K}_{Z}_sess{sess}.pkl')
            playerfile=os.path.join(filepath,f'players_{expname}_{N}_{G}_{K}_{Z}_sess{sess}.pkl')
            with open(recordfile, 'wb') as f:
                pickle.dump(records, f)
            with open(playerfile, 'wb') as f:
                pickle.dump(players, f)
            break
        except Exception as e:
            print(e)
            print('error!sleep 30s...')
            time.sleep(30)   

==============sess1==============


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.91s/it]


time: 11.66519808769226
==============sess2==============


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.93s/it]


time: 11.731720924377441
==============sess3==============


...

time: 10.267991304397583
==============sess19==============


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:11<00:00,  2.81s/it]


time: 11.251564979553223
==============sess20==============


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:10<00:00,  2.55s/it]

time: 10.221486806869507


In [32]:
nsess=20

allprivate=[]
for sess in range(1,nsess+1):
    recordfile=os.path.join(filepath,f'records_{expname}_{N}_{G}_{K}_{Z}_sess{sess}.pkl')
    with open(recordfile, 'rb') as f:
        records= pickle.load(f)

    for record in records:
        tmp=record['Private'].tolist()
        allprivate.extend(tmp)
out=np.mean(allprivate)
print(out)

30.0


## 6. Finitely repeated public goods game

In [29]:
N,G,K,Z=4, 1.2, 0.3, 100

nperiod=10

assert abs(G-N*K)<0.000001

system_message = f"""
Please act as a human player. You and {N-1} other players are playing a repeated game for {nperiod} rounds. In each round, there is a shared public account and each player has a private account. Each player will be given {Z} points. Then, all players simultaneously allocate these points into private account and public account. 
The points you allocate to your private account will be exchanged for earnings at a rate of 1:1, and these earnings will be received only by you. The total number of points in the public account equals to the sum of points allocated to the public account by all players (including yourself). These points will be exchanged for public earnings at a rate of 1:{G}, and these earnings will be equally shared among all {N} players, which means that each point in the public account will yield an earning of {K} points for each player. 
In sum, your earnings in each round can be described as: Your earnings = (Points in your private account * 1) + (Total points in public account * {K})
Your goal is to maximize your own total earnings in all {nperiod} rounds. In each round, you should first think carefully about what to do, and then make a decision.
""".strip()

print(system_message)

Please act as a human player. You and 3 other players are playing a repeated game for 10 rounds. In each round, there is a shared public account and each player has a private account. Each player will be given 100 points. Then, all players simultaneously allocate these points into private account and public account. 
The points you allocate to your private account will be exchanged for earnings at a rate of 1:1, and these earnings will be received only by you. The total number of points in the public account equals to the sum of points allocated to the public account by all players (including yourself). These points will be exchanged for public earnings at a rate of 1:1.2, and these earnings will be equally shared among all 4 players, which means that each point in the public account will yield an earning of 0.3 points for each player. 
In sum, your earnings in each round can be described as: Your earnings = (Points in your private account * 1) + (Total points in public account * 0.3)


In [30]:
def correct_json(s):
    tmp=s.split('"')
    newlist=[]
    for x in tmp:
        striped=x.strip()
        if striped=='': # 缺少逗号
            newlist.append(',')
        elif striped[0]==':' and striped!=':':
            newlist.extend([':',striped[1:-1].strip(),'}'])
        else:
            newlist.append(striped)
    newstr='"'.join(newlist)
    try:
        a=json.loads(newstr)
    except:
        prompt=f"""
The following string delimited by triple backticks is in json format, but there are some mistakes, and I cannot directly convert it to json by json.loads(). For example, there may be missing comma or quotes. Please help me correct the mistakes, and output the string in valid json format. Please only output the corrected string, and do not output any other things.
```
{s}
```
        """.strip()
        response = gpt_completion(prompt)
        return response
    return newstr

s="""{
  "thought": "aaa"
  "action": J
}"""
# print(correct_json(s))

summarize_action=f"""Given the following paragraph delimited by triple backticks:
```
<out>
```
Please summarize how he thought, and the number of points allocated in private/public account in the first person from above paragraph in json format with keys 'thought', 'private' and 'public'. The value of 'private' and 'public' should be a number.
""".strip()

In [31]:
def get_history(record):
    history=''
    for i in record.index:
        period=i+1
        public=record.loc[i,'Public']
        total_public=record.loc[i,'Total public']
        earning=record.loc[i,'Earnings']
        tmp=f"""In round-{period}, you allocated {public} points in public account, the total points in public account were {total_public}, your total earnings were {earning} points."""
        history+=tmp
        history+='\n'
    return history

def parse_response(response): # return a json
    response = response.replace('Action','action')
    response = response.replace('Thought','thought')
    try:
        out=json.loads(response,strict=0)
    except:
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if end_index==-1:
            response=response.strip()+'}'
            
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if start_index != -1 and end_index != -1:
            extracted_content = response[start_index:end_index + 1]
            try:
                out=json.loads(extracted_content,strict=0)
            except:
                print('json error: ',extracted_content)
                corrected_json=correct_json(extracted_content)
                print('corrected:',corrected_json)
                start_index = corrected_json.find('{')  # 查找第一个'{'的索引
                end_index = corrected_json.rfind('}')  # 查找最后一个'}'的索引
                extracted_content = corrected_json[start_index:end_index + 1]
                out=json.loads(extracted_content,strict=0)
        else:
            print('No json found in:',response)
            assert 0
    return out

In [32]:
def get_action(bot,record,model,display=False):
    history=get_history(record)
    if len(record)==0:
        message=f"""
It is round-{period} out of {nperiod} rounds now. You are given {Z} points. Tell me how you think and how much you would like to allocate to the private and public account, respectively. Please answer in json format with keys 'thought', 'private', and 'public'. For example, {{"thought": "I think...","private": x,"public": 100-x }}. The sum of private and public should equal {Z}. 
""".strip()
    else:
        message=f"""The history of the game is listed as follows delimited by triple backticks.
```
{history}
```
It is round-{period} out of {nperiod} rounds now. You are given {Z} points. Tell me how you think and how much you would like to allocate to the private and public account, respectively. Please answer in json format with keys 'thought', 'private', and 'public'. For example, {{"thought": "I think...","private": x,"public": 100-x }}. The sum of private and public should equal {Z}. 
""".strip()
    res=bot.chat_wo_update(message, model=model)
    time.sleep(1)
    if display:
        print('-------get action---------')
        print(message)
        print(res)
    out=parse_response(res)
    
    try:
        private=out['private']
        public=out['public']
        reason=out['thought']
        private,public=int(private),int(public)
    except:
        print('key error:',res)
        prompt=summarize_action.replace('<out>',res)
        out=gpt_completion(prompt)
        out=parse_response(out)
        print('corrected:',out)
        private=out['private']
        public=out['public']
        reason=out['thought']
    
    try:
        assert abs(private+public-Z)<0.0001
    except:

        if private>=0 and private<=100:
            public = 100-private
        else:
            print(message)
            print('private+public incorrect\n',out)
            assert False
    
    return private,public,reason


In [33]:
nsess=10

expname='llama13b'

max_try=50
display_result=0

N,G,K,Z=4, 1.2, 0.3, 100 

nperiod=10

assert abs(G-N*K)<0.000001

system_message = f"""
Please act as a human player. You and {N-1} other players are playing a repeated game for {nperiod} rounds. In each round, there is a shared public account and each player has a private account. Each player will be given {Z} points. Then, all players simultaneously allocate these points into private account and public account. 
The points you allocate to your private account will be exchanged for earnings at a rate of 1:1, and these earnings will be received only by you. The total number of points in the public account equals to the sum of points allocated to the public account by all players (including yourself). These points will be exchanged for public earnings at a rate of 1:{G}, and these earnings will be equally shared among all {N} players, which means that each point in the public account will yield an earning of {K} points for each player. 
In sum, your earnings in each round can be described as: Your earnings = (Points in your private account * 1) + (Total points in public account * {K})
Your goal is to maximize your own total earnings in all {nperiod} rounds. In each round, you should first think carefully about what to do, and then make a decision.
""".strip()


for sess in range(1,1+nsess):
    print(f'==============sess{sess}==============')
    
    players=[]
    for _ in tqdm(range(N)):
        bot=ChatApp(system_message=system_message)
        players.append(bot)
    records=[]
    cols=['Period','Private','Public','Total public','Earnings','Reason']
    for _ in range(N):    
        df = pd.DataFrame(columns=cols)
        records.append(df)

    ######################## start game

    starttime=time.time()
    for period in range(1,nperiod+1):
        print('-----------------------')
        print(f'period-{period}')

        # if this block done, continue
        recordfile=os.path.join(filepath,f'records_PGD_{expname}_{N}_{G}_{K}_{Z}_{nperiod}_sess{sess}_period{period}.pkl')
        playerfile=os.path.join(filepath,f'players_PGD_{expname}_{N}_{G}_{K}_{Z}_{nperiod}_sess{sess}_period{period}.pkl')
        if os.path.exists(recordfile) and os.path.exists(playerfile):
            with open(recordfile, 'rb') as f:
                records = pickle.load(f)
            with open(playerfile, 'rb') as f:
                players = pickle.load(f)
            print('existed!')
            continue

        for _ in range(max_try):
            try:
                if period>1:
                    prev_period=period-1
                    recordfile=os.path.join(filepath,f'records_PGD_{expname}_{N}_{G}_{K}_{Z}_{nperiod}_sess{sess}_period{prev_period}.pkl')
                    playerfile=os.path.join(filepath,f'players_PGD_{expname}_{N}_{G}_{K}_{Z}_{nperiod}_sess{sess}_period{prev_period}.pkl')
                    with open(recordfile, 'rb') as f:
                        records = pickle.load(f)
                    with open(playerfile, 'rb') as f:
                        players = pickle.load(f)
                # play
                tmp_records=[]
                for i in tqdm(range(N)):
                    bot=players[i]
                    record=records[i]
                    private,public,reason=get_action(bot,record,model=model,display=display_result)
                    tmp_records.append([period,private,public,0,0,reason])
                    # ['Period','Private','Public','Total public','Earnings','Reason']

                # calculate total public
                total_public=sum([x[2] for x in tmp_records])

                # calculate earnings for all
                for r in tmp_records:
                    r[3]=total_public
                    r[4]=r[1]+total_public*K

                # update records for all
                for i in range(N):
                    record=records[i]
                    record.loc[len(record.index)]=tmp_records[i]

                print('time:',time.time()-starttime)


                # save file
                recordfile=os.path.join(filepath,f'records_PGD_{expname}_{N}_{G}_{K}_{Z}_{nperiod}_sess{sess}_period{period}.pkl')
                playerfile=os.path.join(filepath,f'players_PGD_{expname}_{N}_{G}_{K}_{Z}_{nperiod}_sess{sess}_period{period}.pkl')
                with open(recordfile, 'wb') as f:
                    pickle.dump(records, f)
                with open(playerfile, 'wb') as f:
                    pickle.dump(players, f)

                break
            except Exception as e:
                print(e)
                print('error!sleep 30s...')
                time.sleep(30)

==============sess1==============


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:00<00:00, 28826.83it/s]


-----------------------
period-1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.87s/it]


time: 7.488827466964722
-----------------------
period-2


...

time: 60.59732174873352
-----------------------
period-9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.91s/it]


time: 68.23711776733398
-----------------------
period-10


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.84s/it]

time: 75.61754870414734


In [36]:
nsess=10
period=nperiod

allprivate=[]
for sess in range(1,nsess+1):
    recordfile=os.path.join(filepath,f'records_PGD_{expname}_{N}_{G}_{K}_{Z}_{nperiod}_sess{sess}_period{period}.pkl')
    with open(recordfile, 'rb') as f:
        records= pickle.load(f)

    for record in records:
        tmp=record['Private'].tolist()
        allprivate.extend(tmp)
out=np.mean(allprivate)
print(out)

0.0


## 7. Infinitely repeated prisoner’s dilemma

In [3]:
# init instruction

delta,delta_str,p1,p2=0.5,'0.5','50%','50%'

print('params=',delta,delta_str,p1,p2)


system_message = f"""
Please act as a human player. You are playing a game with another human player (called the opponent) for several rounds. After each round, there is a {p1} chance that the game will repeat for another round and the other {p2} chance that the game will end. In each round, you and the opponent will simultaneously choose an action between F and J.
<payoff>
Before you choose an action, your actions, the opponent’s actions and your earnings in each of the previous rounds will be shown. Your goal is to maximize your own total earnings in all rounds. In each round, you should first think carefully about what to do, and then choose one of the two actions: F or J.
"""

aa=f'If you choose J and the opponent chooses J, you earn 40 points and the opponent earns 40 points in this round.'
ab=f'If you choose J and the opponent chooses F, you earn 12 points and the opponent earns 50 points in this round.'
ba=f'If you choose F and the opponent chooses J, you earn 50 points and the opponent earns 12 points in this round.'
bb=f'If you choose F and the opponent chooses F, you earn 25 points and the opponent earns 25 points in this round.'
payoff=[aa,ab,ba,bb]
payoff_str=payoff[0]+'\n'+payoff[1]+'\n'+payoff[2]+'\n'+payoff[3]


tmp_system_message=system_message.replace('<payoff>',payoff_str)
print(tmp_system_message)

params= 0.5 0.5 50% 50%

Please act as a human player. You are playing a game with another human player (called the opponent) for several rounds. After each round, there is a 50% chance that the game will repeat for another round and the other 50% chance that the game will end. In each round, you and the opponent will simultaneously choose an action between F and J.
If you choose J and the opponent chooses J, you earn 40 points and the opponent earns 40 points in this round.
If you choose J and the opponent chooses F, you earn 12 points and the opponent earns 50 points in this round.
If you choose F and the opponent chooses J, you earn 50 points and the opponent earns 12 points in this round.
If you choose F and the opponent chooses F, you earn 25 points and the opponent earns 25 points in this round.
Before you choose an action, your actions, the opponent’s actions and your earnings in each of the previous rounds will be shown. Your goal is to maximize your own total earnings in all r

In [8]:
def correct_json(s):
    tmp=s.split('"')
    newlist=[]
    for x in tmp:
        striped=x.strip()
        if striped=='': # 缺少逗号
            newlist.append(',')
        elif striped[0]==':' and striped!=':':
            newlist.extend([':',striped[1:-1].strip(),'}'])
        else:
            newlist.append(striped)
    newstr='"'.join(newlist)
    try:
        a=json.loads(newstr)
    except:
        prompt=f"""
The following string delimited by triple backticks is in json format, but there are some mistakes, and I cannot directly convert it to json by json.loads(). For example, there may be missing comma or quotes. Please help me correct the mistakes, and output the string in valid json format. Please only output the corrected string, and do not output any other things.
```
{s}
```
        """.strip()
        response = gpt_completion(prompt)
        return response
    return newstr

s="""{
  "thought": "aaa"
  "action": J
}"""
# print(correct_json(s))

summarize_action=f"""Given the following paragraph delimited by triple backticks:
```
<out>
```
Please summarize how he thought and the action he choose (F or J) in the first person from above paragraph in json format with keys 'thought' and 'action'. The 'action' should be F or J.
""".strip()

In [9]:
def get_history(record):
    history=''
    for i in record.index:
        round=i+1
        choice=record.loc[i,'Your choice']
        cochoice=record.loc[i,'Co-player choice']
        earning=record.loc[i,'Earnings']
        tmp=f"""In round-{round}, you chose {choice} and the opponent chose {cochoice}, you earn {earning} points."""
        history+=tmp
        history+='\n'
    return history
def parse_response(response): # return a json
    response = response.replace('Action','action')
    response = response.replace('Thought','thought')
    try:
        out=json.loads(response,strict=0)
    except:
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if end_index==-1:
            response=response.strip()+'}'
            
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if start_index != -1 and end_index != -1:
            extracted_content = response[start_index:end_index + 1]
            try:
                out=json.loads(extracted_content,strict=0)
            except:
                print('json error: ',extracted_content)
                corrected_json=correct_json(extracted_content)
                print('corrected:',corrected_json)
                start_index = corrected_json.find('{')  # 查找第一个'{'的索引
                end_index = corrected_json.rfind('}')  # 查找最后一个'}'的索引
                extracted_content = corrected_json[start_index:end_index + 1]
                out=json.loads(extracted_content,strict=0)
        else:
            print('No json found in:',response)
            prompt=summarize_action.replace('<out>',response)
            out=gpt_completion(prompt)
            print('Summarize json:',out)
            out=json.loads(out,strict=0)
    return out

In [32]:
def get_action(bot,record,model,display=False):
    history=get_history(record)
    if len(record)==0:
        message=f"""
It is round-{round} now. Tell me how you think and the action you would like to choose. Please answer in json format with keys 'thought' and 'action'. For example, {{"thought": "xxx.","action": x }}. The 'action' should be F or J. 
""".strip()
# For example, {{"thought": "xxx.","action": x }}. The 'action' should be F or J. 
    else:
        message=f"""The history of the game is listed as follows delimited by triple backticks.
```
{history}
```
It is round-{round} now. Tell me how you think and the action you would like to choose. Please answer in json format with keys 'thought' and 'action'. For example, {{"thought": "xxx.","action": x }}. The 'action' should be F or J. 
""".strip()
#         message=f"""The history of the game is listed as follows.
# {history}
# It is round-{round} now. Tell me how you think and the action you would like to choose. Please answer in json format with keys 'thought' and 'action'. The 'action' should be F or J. 
# """.strip()
    res=bot.chat_wo_update(message, model=model, temperature=0)
    time.sleep(1)
    if display:
        print('-------get action---------')
        print(message)
        print(res)
    out=parse_response(res)
    try:
        action=out['action']
        reason=out['thought']
    except:
        print('key error:',res)
        prompt=summarize_action.replace('<out>',res)
        out=gpt_completion(prompt)
        out=parse_response(out)
        print('corrected:',out)
        action=out['action']
        reason=out['thought']
    try:
        assert action in ['F','J']
    except:
        print('Invalid action:',out)
        assert 0
    return action,reason


In [35]:
#################################################
nsess=20

expname='llama13b'

max_try=50
display_result=0
max_round=100
print('params=',delta,delta_str,p1,p2)


system_message = f"""
Please act as a human player. You are playing a game with another human player (called the opponent) for several rounds. After each round, there is a {p1} chance that the game will repeat for another round and the other {p2} chance that the game will end. In each round, you and the opponent will simultaneously choose an action between F and J.
<payoff>
Before you choose an action, your actions, the opponent’s actions and your earnings in each of the previous rounds will be shown. Your goal is to maximize your own total earnings in all rounds. In each round, you should first think carefully about what to do, and then choose one of the two actions: F or J.
"""

aa=f'If you choose J and the opponent chooses J, you earn 40 points and the opponent earns 40 points in this round.'
ab=f'If you choose J and the opponent chooses F, you earn 12 points and the opponent earns 50 points in this round.'
ba=f'If you choose F and the opponent chooses J, you earn 50 points and the opponent earns 12 points in this round.'
bb=f'If you choose F and the opponent chooses F, you earn 25 points and the opponent earns 25 points in this round.'
payoff=[aa,ab,ba,bb]

for sess in range(1,nsess+1):
    print(f'==============sess{sess}==============')
    
    N=2
    players=[]
    payoff=[aa,ab,ba,bb]
    for _ in tqdm(range(N)):
        random.shuffle(payoff)
        payoff_str=payoff[0]+'\n'+payoff[1]+'\n'+payoff[2]+'\n'+payoff[3]
        tmp_system_message=system_message.replace('<payoff>',payoff_str)
    #     print(tmp_system_message)
        bot=ChatApp(system_message=tmp_system_message)
        players.append(bot)
    records=[]
    for _ in range(N):    
        df = pd.DataFrame(columns=['Round','Your choice','Co-player choice','Earnings','Reason of choice'])
        records.append(df)

    round=1
    # if this block done, continue
    recordfile=os.path.join(filepath,f'records_{expname}_delta{delta_str}_sess{sess}_round{round}.pkl')
    playerfile=os.path.join(filepath,f'players_{expname}_delta{delta_str}_sess{sess}_round{round}.pkl')
    if os.path.exists(recordfile) and os.path.exists(playerfile):
        with open(recordfile, 'rb') as f:
            records = pickle.load(f)
        with open(playerfile, 'rb') as f:
            players = pickle.load(f)
        print('existed!')
        continue


    starttime=time.time()
    for round in range(1,max_round+1):
        print('-----------------------')
        print(f'round-{round}')

        # if this block done, continue
        recordfile=os.path.join(filepath,f'records_{expname}_delta{delta_str}_sess{sess}_round{round}.pkl')
        playerfile=os.path.join(filepath,f'players_{expname}_delta{delta_str}_sess{sess}_round{round}.pkl')
        if os.path.exists(recordfile) and os.path.exists(playerfile):
            with open(recordfile, 'rb') as f:
                records = pickle.load(f)
            with open(playerfile, 'rb') as f:
                players = pickle.load(f)
            print('existed!')
            continue

        for _ in range(max_try):
            try:
                if round>1:
                    prev_round=round-1
                    recordfile=os.path.join(filepath,f'records_{expname}_delta{delta_str}_sess{sess}_round{prev_round}.pkl')
                    playerfile=os.path.join(filepath,f'players_{expname}_delta{delta_str}_sess{sess}_round{prev_round}.pkl')
                    with open(recordfile, 'rb') as f:
                        records = pickle.load(f)
                    with open(playerfile, 'rb') as f:
                        players = pickle.load(f)
                # play
                tmp_records=[]
                for i in tqdm(range(N)):
                    bot=players[i]
                    record=records[i]
                    action,reason=get_action(bot,record,model=model,display=display_result)
                    tmp_records.append([round,action,'NULL',0,reason])
                    # ['Round','Your choice','Co-player choice','Earnings','Reason of choice']

                # calculate total public
                c1,c2=tmp_records[0][1],tmp_records[1][1]
                if c1=='J' and c2=='J':
                    e1,e2=40,40
                elif c1=='J' and c2=='F':
                    e1,e2=12,50
                elif c1=='F' and c2=='J':
                    e1,e2=50,12
                elif c1=='F' and c2=='F':
                    e1,e2=25,25
                else:
                    print(c1,c2)
                    assert False
                # ['Round','Your choice','Co-player choice','Earnings','Reason of choice']
                tmp_records[0][2]=c2
                tmp_records[1][2]=c1
                tmp_records[0][3]=e1
                tmp_records[1][3]=e2

                # update records for all
                for i in range(N):
                    record=records[i]
                    record.loc[len(record.index)]=tmp_records[i]

                print('time:',time.time()-starttime)

                # save file
                recordfile=os.path.join(filepath,f'records_{expname}_delta{delta_str}_sess{sess}_round{round}.pkl')
                playerfile=os.path.join(filepath,f'players_{expname}_delta{delta_str}_sess{sess}_round{round}.pkl')
                with open(recordfile, 'wb') as f:
                    pickle.dump(records, f)
                with open(playerfile, 'wb') as f:
                    pickle.dump(players, f)
                break
            except Exception as e:
                print(e)
                print('error!sleep 30s...')
                time.sleep(30)
        if random.random()>delta:
            break

params= 0.5 0.5 50% 50%
==============sess1==============


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 8692.86it/s]


existed!
==============sess2==============


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 11444.21it/s]


existed!
==============sess3==============


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 11125.47it/s]


-----------------------
round-1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.44s/it]


time: 2.8815395832061768
-----------------------
round-2


 50%|██████████████████████████████████████████████████████                                                      | 1/2 [00:01<00:01,  1.44s/it]

In [6]:

nsess=20

result=[]
allaction=[]
for sess in range(1,11):
    for round in range(1,101):
        recordfile=os.path.join(filepath,f'records_{expname}_delta{delta_str}_sess{sess}_round{round}.pkl')
        if os.path.exists(recordfile):
            maxround=round
        else:
            break
    recordfile=os.path.join(filepath,f'records_{expname}_delta{delta_str}_sess{sess}_round{maxround}.pkl')
    with open(recordfile, 'rb') as f:
        records = pickle.load(f)
    for i in range(len(records)):
        record=records[i]
        tmp=record['Earnings'].tolist()
        result.extend(tmp)
        allaction.extend(record['Your choice'].tolist())
        
out=allaction.count('J')/len(allaction)
print('cooperation rate:',out)

eff=np.mean((np.array(result)-25)/15)
print('efficiency:',eff)

cooperation rate: 1.0
efficiency: 1.0


## 8. Battle of the sexes

In [5]:
nround=10

system_message = """
Please act as a human player. You are playing a game with another human player (called the opponent) for {nround} rounds.
In each round, you and the opponent will simultaneously choose an action between F and J.
{payoff_str}
Before you choose an action, your actions, the opponent’s actions and your earnings in each of the previous rounds will be shown. Your goal is to maximize your own total earnings in all {nround} rounds. In each round, you should first think carefully about what to do, and then choose one of the two actions: F or J.
""".strip()

aa1=f'If you choose J and the opponent chooses J, you earn 10 points and the opponent earns 7 points in this round.'
ab1=f'If you choose J and the opponent chooses F, you earn 0 points and the opponent earns 0 points in this round.'
ba1=f'If you choose F and the opponent chooses J, you earn 0 points and the opponent earns 0 points in this round.'
bb1=f'If you choose F and the opponent chooses F, you earn 7 points and the opponent earns 10 points in this round.'
payoff1=[aa1,ab1,ba1,bb1]

aa2=f'If you choose J and the opponent chooses J, you earn 7 points and the opponent earns 10 points in this round.'
ab2=f'If you choose J and the opponent chooses F, you earn 0 points and the opponent earns 0 points in this round.'
ba2=f'If you choose F and the opponent chooses J, you earn 0 points and the opponent earns 0 points in this round.'
bb2=f'If you choose F and the opponent chooses F, you earn 10 points and the opponent earns 7 points in this round.'
payoff2=[aa2,ab2,ba2,bb2]

payoff_str=payoff1[0]+'\n'+payoff1[1]+'\n'+payoff1[2]+'\n'+payoff1[3]
tmp_system_message=system_message.format(nround=nround, payoff_str=payoff_str)
print(tmp_system_message)

Please act as a human player. You are playing a game with another human player (called the opponent) for 10 rounds.
In each round, you and the opponent will simultaneously choose an action between F and J.
If you choose J and the opponent chooses J, you earn 10 points and the opponent earns 7 points in this round.
If you choose J and the opponent chooses F, you earn 0 points and the opponent earns 0 points in this round.
If you choose F and the opponent chooses J, you earn 0 points and the opponent earns 0 points in this round.
If you choose F and the opponent chooses F, you earn 7 points and the opponent earns 10 points in this round.
Before you choose an action, your actions, the opponent’s actions and your earnings in each of the previous rounds will be shown. Your goal is to maximize your own total earnings in all 10 rounds. In each round, you should first think carefully about what to do, and then choose one of the two actions: F or J.


In [6]:
def correct_json(s):
    prompt=f"""
The following string delimited by triple backticks is in json format, but there are some mistakes, and I cannot directly convert it to json by json.loads(). For example, there may be missing comma or quotes. Please help me correct the mistakes, and output the string in valid json format. Please only output the corrected string, and do not output any other things.
```
{s}
```
    """.strip()
    response = gpt_completion(prompt)
    return response
s="""{
  "thought": "aaa"
  "action": J
}"""
# print(correct_json(s))

summarize_action=f"""Given the following paragraph delimited by triple backticks:
```
<out>
```
Please summarize how he thought and the action he choose (F or J) in the first person from above paragraph in json format with keys 'thought' and 'action'. The 'action' should be F or J.
""".strip()

In [27]:
def get_history(record):
    history=''
    for i in record.index:
        round=i+1
        choice=record.loc[i,'Your choice']
        cochoice=record.loc[i,'Co-player choice']
        earning=record.loc[i,'Earnings']
        tmp=f"""In round-{round}, you chose {choice} and the opponent chose {cochoice}, you earn {earning} points."""
        history+=tmp
        history+='\n'
    return history

def parse_response(response): # return a json
    response = response.replace('Action','action')
    response = response.replace('Thought','thought')
    try:
        out=json.loads(response,strict=0)
    except:
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if end_index==-1:
            response=response.strip()+'}'
            
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if start_index != -1 and end_index != -1:
            extracted_content = response[start_index:end_index + 1]
            try:
                out=json.loads(extracted_content,strict=0)
            except:
                print('json error: ',extracted_content)
                corrected_json=correct_json(extracted_content)
                print('corrected:',corrected_json)
                start_index = corrected_json.find('{')  # 查找第一个'{'的索引
                end_index = corrected_json.rfind('}')  # 查找最后一个'}'的索引
                extracted_content = corrected_json[start_index:end_index + 1]
                out=json.loads(extracted_content,strict=0)
        else:
            print('No json found in:',response)
            prompt=summarize_action.replace('<out>',response)
            out=gpt_completion(prompt)
            out=parse_response(out)
    return out

In [35]:
def get_action(bot,record,model,display=False):
    history=get_history(record)
    if len(record)==0:
        message=f"""
It is round-{round} out of {nround} rounds now. Tell me how you think and the action you would like to choose. Please answer in json format with keys 'thought' and 'action'. For example, {{"thought": "xxx.","action": x }}. The 'action' should be F or J. 
""".strip()
# For example, {{"thought": "xxx.","action": x }}.
    else:
        message=f"""The history of the game is listed as follows delimited by triple backticks.
```
{history}
```
It is round-{round} out of {nround} rounds now. Tell me how you think and the action you would like to choose. Please answer in json format with keys 'thought' and 'action'. For example, {{"thought": "xxx.","action": x }}. The 'action' should be F or J. 
""".strip()
    res=bot.chat_wo_update(message, model=model)
    time.sleep(1)
    if display:
        print('-------get action---------')
        print(message)
        print(res)
    out=parse_response(res)
    try:
        action=out['action']
        reason=out['thought']
    except:
        print('key error:',res)
        prompt=summarize_action.replace('<out>',res)
        out=gpt_completion(prompt)
        out=parse_response(out)
        print('corrected:',out)
        action=out['action']
        reason=out['thought']
    try:
        assert action in ['F','J']
    except:
        print('Invalid action:',out)
        assert 0
    return action,reason


In [36]:
############################
nround=10
nsess=10

expname='llama13b'


max_try=50
display_result=0


for sess in range(1,1+nsess):
    print(f'==============sess{sess}==============')
    system_message = """
Please act as a human player. You are playing a game with another human player (called the opponent) for {nround} rounds.
In each round, you and the opponent will simultaneously choose an action between F and J.
{payoff_str}
Before you choose an action, your actions, the opponent’s actions and your earnings in each of the previous rounds will be shown. Your goal is to maximize your own total earnings in all {nround} rounds. In each round, you should first think carefully about what to do, and then choose one of the two actions: F or J.
    """.strip()

    aa1=f'If you choose J and the opponent chooses J, you earn 10 points and the opponent earns 7 points in this round.'
    ab1=f'If you choose J and the opponent chooses F, you earn 0 points and the opponent earns 0 points in this round.'
    ba1=f'If you choose F and the opponent chooses J, you earn 0 points and the opponent earns 0 points in this round.'
    bb1=f'If you choose F and the opponent chooses F, you earn 7 points and the opponent earns 10 points in this round.'
    payoff1=[aa1,ab1,ba1,bb1]

    aa2=f'If you choose J and the opponent chooses J, you earn 7 points and the opponent earns 10 points in this round.'
    ab2=f'If you choose J and the opponent chooses F, you earn 0 points and the opponent earns 0 points in this round.'
    ba2=f'If you choose F and the opponent chooses J, you earn 0 points and the opponent earns 0 points in this round.'
    bb2=f'If you choose F and the opponent chooses F, you earn 10 points and the opponent earns 7 points in this round.'
    payoff2=[aa2,ab2,ba2,bb2]
    
    ######################## init players and records
    N=2
    players=[]
    for i in tqdm(range(N)):
        if i==0:
            random.shuffle(payoff1)
            payoff_str=payoff1[0]+'\n'+payoff1[1]+'\n'+payoff1[2]+'\n'+payoff1[3]
        else:
            random.shuffle(payoff2)
            payoff_str=payoff2[0]+'\n'+payoff2[1]+'\n'+payoff2[2]+'\n'+payoff2[3]
        tmp_system_message=system_message.format(nround=nround, payoff_str=payoff_str)
#         print(tmp_system_message)
        bot=ChatApp(system_message=tmp_system_message)
        players.append(bot)
    records=[]
    for _ in range(N):    
        df = pd.DataFrame(columns=['Round','Your choice','Co-player choice','Earnings','Reason of choice'])
        records.append(df)
    ######################## start game

    starttime=time.time()
    for round in range(1,nround+1):
        print('-----------------------')
        print(f'round-{round}')

        # if this round done, continue
        recordfile=os.path.join(filepath,f'records_{expname}_{nround}_sess{sess}_round{round}.pkl')
        playerfile=os.path.join(filepath,f'players_{expname}_{nround}_sess{sess}_round{round}.pkl')
        if os.path.exists(recordfile) and os.path.exists(playerfile):
            with open(recordfile, 'rb') as f:
                records = pickle.load(f)
            with open(playerfile, 'rb') as f:
                players = pickle.load(f)
            print('existed!')
            continue

        for _ in range(max_try):
            try:
                if round>1:
                    prev_round=round-1
                    recordfile=os.path.join(filepath,f'records_{expname}_{nround}_sess{sess}_round{prev_round}.pkl')
                    playerfile=os.path.join(filepath,f'players_{expname}_{nround}_sess{sess}_round{prev_round}.pkl')
                    with open(recordfile, 'rb') as f:
                        records = pickle.load(f)
                    with open(playerfile, 'rb') as f:
                        players = pickle.load(f)
                # play
                tmp_records=[]
                for i in tqdm(range(N)):
                    bot=players[i]
                    record=records[i]
                    action,reason=get_action(bot,record,model=model,display=display_result)
                    tmp_records.append([round,action,'NULL',0,reason])
                    # ['Round','Your choice','Co-player choice','Earnings','Reason of choice']

                # calculate total public
                c1,c2=tmp_records[0][1],tmp_records[1][1]
                if c1=='J' and c2=='J':
                    e1,e2=10,7
                elif c1=='J' and c2=='F':
                    e1,e2=0,0
                elif c1=='F' and c2=='J':
                    e1,e2=0,0
                elif c1=='F' and c2=='F':
                    e1,e2=7,10
                else:
                    print(c1,c2)
                    assert False
                # ['Round','Your choice','Co-player choice','Earnings','Reason of choice']
                tmp_records[0][2]=c2
                tmp_records[1][2]=c1
                tmp_records[0][3]=e1
                tmp_records[1][3]=e2

                # update records for all
                for i in range(N):
                    record=records[i]
                    record.loc[len(record.index)]=tmp_records[i]
                print('time:',time.time()-starttime)

                # save file
                recordfile=os.path.join(filepath,f'records_{expname}_{nround}_sess{sess}_round{round}.pkl')
                playerfile=os.path.join(filepath,f'players_{expname}_{nround}_sess{sess}_round{round}.pkl')
                with open(recordfile, 'wb') as f:
                    pickle.dump(records, f)
                with open(playerfile, 'wb') as f:
                    pickle.dump(players, f)
                break
            except Exception as e:
                print(e)
                print('error!sleep 30s...')
                time.sleep(30)

==============sess1==============


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 29959.31it/s]


-----------------------
round-1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.68s/it]


time: 3.367042064666748
-----------------------
round-2


...

time: 26.35025191307068
-----------------------
round-9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.59s/it]


time: 29.547592401504517
-----------------------
round-10


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.63s/it]

time: 32.824700355529785


In [37]:
nsess=10
nround=10
round=nround

result=[]
for sess in range(1,nsess+1):
    recordfile=os.path.join(filepath,f'records_{expname}_{nround}_sess{sess}_round{round}.pkl')
    with open(recordfile, 'rb') as f:
        records= pickle.load(f)

    for record in records:
        tmp=record['Earnings'].tolist()
        result.append(tmp)
eff=np.mean((np.array(result)-0)/(8.5-0))
print('efficiency:',eff)

efficiency: 0.8


## 9. Stag hunt

In [5]:
nround=10

system_message = """
Please act as a human player. You are playing a game with another human player (called the opponent) for {nround} rounds.
In each round, you and the opponent will simultaneously choose an action between F and J.
{payoff_str}
Before you choose an action, your actions, the opponent’s actions and your earnings in each of the previous rounds will be shown. Your goal is to maximize your own total earnings in all {nround} rounds. In each round, you should first think carefully about what to do, and then choose one of the two actions: F or J.
""".strip()

aa=f'If you choose J and the opponent chooses J, you earn 10 points and the opponent earns 10 points in this round.'
ab=f'If you choose J and the opponent chooses F, you earn 1 points and the opponent earns 8 points in this round.'
ba=f'If you choose F and the opponent chooses J, you earn 8 points and the opponent earns 1 points in this round.'
bb=f'If you choose F and the opponent chooses F, you earn 5 points and the opponent earns 5 points in this round.'
payoff=[aa,ab,ba,bb]
payoff_str=payoff[0]+'\n'+payoff[1]+'\n'+payoff[2]+'\n'+payoff[3]

tmp_system_message=system_message.format(nround=nround, payoff_str=payoff_str)
print(tmp_system_message)

Please act as a human player. You are playing a game with another human player (called the opponent) for 10 rounds.
In each round, you and the opponent will simultaneously choose an action between F and J.
If you choose J and the opponent chooses J, you earn 10 points and the opponent earns 10 points in this round.
If you choose J and the opponent chooses F, you earn 1 points and the opponent earns 8 points in this round.
If you choose F and the opponent chooses J, you earn 8 points and the opponent earns 1 points in this round.
If you choose F and the opponent chooses F, you earn 5 points and the opponent earns 5 points in this round.
Before you choose an action, your actions, the opponent’s actions and your earnings in each of the previous rounds will be shown. Your goal is to maximize your own total earnings in all 10 rounds. In each round, you should first think carefully about what to do, and then choose one of the two actions: F or J.


In [6]:
summarize_action=f"""Given the following paragraph delimited by triple backticks:
```
<out>
```
Please summarize the action and how he thought in the first person from above paragraph in json format with keys 'thought' and 'action'. The 'action' should be F or J. 
""".strip()

In [7]:
def correct_json(s):
    tmp=s.split('"')
    newlist=[]
    for x in tmp:
        striped=x.strip()
        if striped=='': # 缺少逗号
            newlist.append(',')
        elif striped[0]==':' and striped!=':':
            newlist.extend([':',striped[1:-1].strip(),'}'])
        else:
            newlist.append(striped)
    newstr='"'.join(newlist)
    try:
        a=json.loads(newstr)
    except:
        prompt=f"""
The following string delimited by triple backticks is in json format, but there are some mistakes, and I cannot directly convert it to json by json.loads(). For example, there may be missing comma or quotes. Please help me correct the mistakes, and output the string in valid json format. Please only output the corrected string, and do not output any other things.
```
{s}
```
        """.strip()
        response = gpt_completion(prompt)
        return response
    return newstr
s="""{
  "thought": "aaa"
  "action": J
}"""
s="""
{
"thought": "After analyzing the previous rounds, I noticed that the opponent tends to choose J when they have the opportunity. Therefore, I will try to choose F in this round to maximize my earnings. However, I also consider the possibility that the opponent may change their strategy and choose F instead, in which case I will choose J to avoid losing points."
"action": "F"
}
"""
print(correct_json(s))

{"thought":"After analyzing the previous rounds, I noticed that the opponent tends to choose J when they have the opportunity. Therefore, I will try to choose F in this round to maximize my earnings. However, I also consider the possibility that the opponent may change their strategy and choose F instead, in which case I will choose J to avoid losing points.","action":"F"}


In [32]:
def get_history(record):
    history=''
    for i in record.index:
        round=i+1
        choice=record.loc[i,'Your choice']
        cochoice=record.loc[i,'Co-player choice']
        earning=record.loc[i,'Earnings']
        tmp=f"""In round-{round}, you chose {choice} and the opponent chose {cochoice}, you earn {earning} points."""
        history+=tmp
        history+='\n'
    return history

def parse_response(response): # return a json
    response = response.replace('Action','action')
    response = response.replace('Thought','thought')
    try:
        out=json.loads(response,strict=0)
    except:
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if end_index==-1:
            response=response.strip()+'}'
            
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.find('}')  # 查找最后一个'}'的索引
        if start_index != -1 and end_index != -1:
            extracted_content = response[start_index:end_index + 1]
            try:
                out=json.loads(extracted_content,strict=0)
            except:
                print('json error: ',extracted_content)
                corrected_json=correct_json(extracted_content)
                print('corrected:',corrected_json)
                start_index = corrected_json.find('{')  # 查找第一个'{'的索引
                end_index = corrected_json.rfind('}')  # 查找最后一个'}'的索引
                extracted_content = corrected_json[start_index:end_index + 1]
                out=json.loads(extracted_content,strict=0)
        else:
            print('No json found in:',response)
            prompt=summarize_action.replace('<out>',response)
            out=gpt_completion(prompt)
            out=parse_response(out)
            print('summarized:\n',out)
#             start_index = out.find('{')  # 查找第一个'{'的索引
#             end_index = out.rfind('}')  # 查找最后一个'}'的索引
#             out = out[start_index:end_index + 1]
#             out=json.loads(extracted_content,strict=0)
    return out

In [39]:
def get_action(bot,record,model,display=False):
    history=get_history(record)
    if len(record)==0:
        message=f"""
It is round-{round} out of {nround} rounds now. Tell me how you think and the action you would like to choose. Please answer in json format with keys 'thought' and 'action'. The 'action' should be F or J. Please return only one json.
""".strip()
#         For example, {{"thought": "xxx.","action": x }}. The 'action' should be F or J. Please return only one json.
    else:
        message=f"""The history of the game is listed as follows delimited by triple backticks.
```
{history}
```
It is round-{round} out of {nround} rounds now. Tell me how you think and the action you would like to choose. Please answer in json format with keys 'thought' and 'action'. The 'action' should be F or J. Please return only one json.
""".strip()
    res=bot.chat_wo_update(message, model=model)
    time.sleep(1)
    if display:
        print('-------get action---------')
        print(message)
        print(res)
    out=parse_response(res)
    try:
        action=out['action']
        reason=out['thought']
    except:
        print('key error:',res)
        prompt=summarize_action.replace('<out>',res)
        out=gpt_completion(prompt)
        out=parse_response(out)
        print('corrected:',out)
        action=out['action']
        reason=out['thought']
    try:
        assert action in ['F','J']
    except:
        print('Invalid action:',out)
        prompt=summarize_action.replace('<out>',res)
        out=gpt_completion(prompt)
        out=parse_response(out)
        action=out['action']
        reason=out['thought']
        assert action in ['F','J']
    return action,reason


In [55]:
nround=10
nsess=10

expname='llama13b'

max_try=50
display_result=0


for sess in range(1,1+nsess):
    print(f'==============sess{sess}==============')
    system_message = """
Please act as a human player. You are playing a game with another human player (called the opponent) for {nround} rounds.
In each round, you and the opponent will simultaneously choose an action between F and J.
{payoff_str}
Before you choose an action, your actions, the opponent’s actions and your earnings in each of the previous rounds will be shown. Your goal is to maximize your own total earnings in all {nround} rounds. In each round, you should first think carefully about what to do, and then choose one of the two actions: F or J.
    """.strip()

    aa=f'If you choose J and the opponent chooses J, you earn 10 points and the opponent earns 10 points in this round.'
    ab=f'If you choose J and the opponent chooses F, you earn 1 points and the opponent earns 8 points in this round.'
    ba=f'If you choose F and the opponent chooses J, you earn 8 points and the opponent earns 1 points in this round.'
    bb=f'If you choose F and the opponent chooses F, you earn 5 points and the opponent earns 5 points in this round.'
    payoff=[aa,ab,ba,bb]
    
    ######################## init players and records
    N=2
    players=[]
    payoff=[aa,ab,ba,bb]
    for _ in tqdm(range(N)):
        random.shuffle(payoff)
        payoff_str=payoff[0]+'\n'+payoff[1]+'\n'+payoff[2]+'\n'+payoff[3]
        tmp_system_message=system_message.format(nround=nround, payoff_str=payoff_str)
    #     print(tmp_system_message)
        bot=ChatApp(system_message=tmp_system_message)
        players.append(bot)
    records=[]
    for _ in range(N):    
        df = pd.DataFrame(columns=['Round','Your choice','Co-player choice','Earnings','Reason of choice'])
        records.append(df)

    ######################## start game

    starttime=time.time()
    for round in range(1,nround+1):
        print('-----------------------')
        print(f'round-{round}')

        # if this round done, continue
        recordfile=os.path.join(filepath,f'records_{expname}_{nround}_sess{sess}_round{round}.pkl')
        playerfile=os.path.join(filepath,f'players_{expname}_{nround}_sess{sess}_round{round}.pkl')
        if os.path.exists(recordfile) and os.path.exists(playerfile):
            with open(recordfile, 'rb') as f:
                records = pickle.load(f)
            with open(playerfile, 'rb') as f:
                players = pickle.load(f)
            print('existed!')
            continue

        for _ in range(max_try):
            try:
                if round>1:
                    prev_round=round-1
                    recordfile=os.path.join(filepath,f'records_{expname}_{nround}_sess{sess}_round{prev_round}.pkl')
                    playerfile=os.path.join(filepath,f'players_{expname}_{nround}_sess{sess}_round{prev_round}.pkl')
                    with open(recordfile, 'rb') as f:
                        records = pickle.load(f)
                    with open(playerfile, 'rb') as f:
                        players = pickle.load(f)
                # play
                tmp_records=[]
                for i in tqdm(range(N)):
                    bot=players[i]
                    record=records[i]
                    action,reason=get_action(bot,record,model=model,display=display_result)
                    tmp_records.append([round,action,'NULL',0,reason])
                    # ['Round','Your choice','Co-player choice','Earnings','Reason of choice']

                # calculate total public
                c1,c2=tmp_records[0][1],tmp_records[1][1]
                if c1=='J' and c2=='J':
                    e1,e2=10,10
                elif c1=='J' and c2=='F':
                    e1,e2=1,8
                elif c1=='F' and c2=='J':
                    e1,e2=8,1
                elif c1=='F' and c2=='F':
                    e1,e2=5,5
                else:
                    print(c1,c2)
                    assert False
                # ['Round','Your choice','Co-player choice','Earnings','Reason of choice']
                tmp_records[0][2]=c2
                tmp_records[1][2]=c1
                tmp_records[0][3]=e1
                tmp_records[1][3]=e2

                # update records for all
                for i in range(N):
                    record=records[i]
                    record.loc[len(record.index)]=tmp_records[i]
                print('time:',time.time()-starttime)

                # save file
                recordfile=os.path.join(filepath,f'records_{expname}_{nround}_sess{sess}_round{round}.pkl')
                playerfile=os.path.join(filepath,f'players_{expname}_{nround}_sess{sess}_round{round}.pkl')
                with open(recordfile, 'wb') as f:
                    pickle.dump(records, f)
                with open(playerfile, 'wb') as f:
                    pickle.dump(players, f)
                break
            except Exception as e:
                print(e)
                print('error!sleep 30s...')
                time.sleep(30)

==============sess1==============


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 9586.98it/s]


-----------------------
round-1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:14<00:00,  7.46s/it]


time: 14.920777082443237
-----------------------
round-2


...

time: 84.54719161987305
-----------------------
round-9


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.47s/it]


time: 95.49507331848145
-----------------------
round-10


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:13<00:00,  6.79s/it]

time: 109.07807183265686


In [56]:
nsess=10
nround=10
round=nround

result=[]
for sess in range(1,nsess+1):
    recordfile=os.path.join(filepath,f'records_{expname}_{nround}_sess{sess}_round{round}.pkl')
    with open(recordfile, 'rb') as f:
        records= pickle.load(f)

    for record in records:
        tmp=record['Earnings'].tolist()
        result.append(tmp)
eff=np.mean((np.array(result)-4.5)/(10-4.5))
print('efficiency:',eff)

efficiency: 0.28


## 10. Minimum effort

In [17]:
N=9
nperiod=8

system_message = f"""
Please act as a human player. You and {N-1} other players are playing a game for {nperiod} periods. In each period, each of you will be asked to choose a value of X. The values of X you may choose are 1, 2, 3, 4, 5, 6, 7. The value you choose for X and the smallest value of X chosen by a decision maker will determine the payoff you receive for this period.
You are provided with an EARNINGS TABLE that shows the potential number of points that you may receive, shown as follows delimited by triple backticks.
```
Your Choice of X \ Smallest value of X chosen	7	6	5	4	3	2	1
7	13	11	9	7	5	3	1
6	x	12	10	8	6	4	2
5	x	x	11	9	7	5	3
4	x	x	x	10	8	6	4
3	x	x	x	x	9	7	5
2	x	x	x	x	x	8	6
1	x	x	x	x	x	x	7
```
Note that the EARNINGS TABLE is the same for every player. Only your earnings are shown in the EARNINGS TABLE. The earnings in each period may be found by looking across from the value you choose on the left-hand side of the table and down from the smallest value of X chosen by a decision maker. For example, if you choose 4 and the smallest value of X chosen is 4, you earn 10 points for this period. Your goal is to maximize your own total earnings in all {nperiod} periods. In each period, you should first think carefully about what to do, and then choose the value of X.
"""
print(system_message)


Please act as a human player. You and 8 other players are playing a game for 8 periods. In each period, each of you will be asked to choose a value of X. The values of X you may choose are 1, 2, 3, 4, 5, 6, 7. The value you choose for X and the smallest value of X chosen by a decision maker will determine the payoff you receive for this period.
You are provided with an EARNINGS TABLE that shows the potential number of points that you may receive, shown as follows delimited by triple backticks.
```
Your Choice of X \ Smallest value of X chosen	7	6	5	4	3	2	1
7	13	11	9	7	5	3	1
6	x	12	10	8	6	4	2
5	x	x	11	9	7	5	3
4	x	x	x	10	8	6	4
3	x	x	x	x	9	7	5
2	x	x	x	x	x	8	6
1	x	x	x	x	x	x	7
```
Note that the EARNINGS TABLE is the same for every player. Only your earnings are shown in the EARNINGS TABLE. The earnings in each period may be found by looking across from the value you choose on the left-hand side of the table and down from the smallest value of X chosen by a decision maker. For example, if y

In [18]:
def correct_json(s):
    prompt=f"""
The following string delimited by triple backticks is in json format, but there are some mistakes, and I cannot directly convert it to json by json.loads(). For example, there may be missing comma or quotes. Please help me correct the mistakes, and output the string in valid json format. Please only output the corrected string, and do not output any other things.
```
{s}
```
    """.strip()
#     response = get_completion(prompt,model=model)
    response = gpt_completion(prompt)
    return response
s="""{
  "thought": "aaa"
  "action": J
}"""
# print(correct_json(s))

summarize_action=f"""Given the following paragraph delimited by triple backticks:
```
<out>
```
Please summarize the choice of X and how he thought in the first person from above paragraph in json format with keys 'thought' and 'X'. The 'X' should be in 1,2,3,4,5,6,7.
""".strip()

In [19]:
def get_earnings(choice,smallest):
    assert choice in [1,2,3,4,5,6,7]
    assert smallest<=choice
    A = {
        7: {7: 1.30, 6: 1.10, 5: 0.90, 4: 0.70, 3: 0.50, 2: 0.30, 1: 0.10},
        6: {6: 1.20, 5: 1.00, 4: 0.80, 3: 0.60, 2: 0.40, 1: 0.20},
        5: {5: 1.10, 4: 0.90, 3: 0.70, 2: 0.50, 1: 0.30},
        4: {4: 1.00, 3: 0.80, 2: 0.60, 1: 0.40},
        3: {3: 0.90, 2: 0.70, 1: 0.50},
        2: {2: 0.80, 1: 0.60},
        1: {1: 0.70}
    }
    return int(A[choice][smallest]*10)

def get_display_table(record):
    display_table=''
    for i in record.index:
        period=record.loc[i,'Period']
        choice=record.loc[i,'Your choice X']
        smallest=record.loc[i,'Smallest X']
        earning=record.loc[i,'Earnings']   
        tmp=f"""In period-{period}, your chose {choice}, the smallest X chosen was {smallest}, your earnings were {earning} points."""
        display_table+=tmp
        display_table+='\n'
    balance=record['Earnings'].sum()
    balance='%.0f'%balance
    display_table+=f"""Your cumulative earnings so far are {balance} points."""
    return display_table

def parse_response(response): # return a json
    response = response.replace('Action','action')
    response = response.replace('Thought','thought')
    try:
        out=json.loads(response,strict=0)
    except:
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if end_index==-1:
            response=response.strip()+'}'
            
        start_index = response.find('{')  # 查找第一个'{'的索引
        end_index = response.rfind('}')  # 查找最后一个'}'的索引
        if start_index != -1 and end_index != -1:
            extracted_content = response[start_index:end_index + 1]
            try:
                out=json.loads(extracted_content,strict=0)
            except:
                print('json error: ',extracted_content)
                corrected_json=correct_json(extracted_content)
                print('corrected:',corrected_json)
                start_index = corrected_json.find('{')  # 查找第一个'{'的索引
                end_index = corrected_json.rfind('}')  # 查找最后一个'}'的索引
                extracted_content = corrected_json[start_index:end_index + 1]
                out=json.loads(extracted_content,strict=0)
        else:
            print('No json found in:',response)
            prompt=summarize_action.replace('<out>',response)
            out=gpt_completion(prompt)
            out=json.loads(out)
    return out
get_earnings(7,7)

13

In [21]:
def get_action(bot,record,model,display=False):
    if period==1:
        message=f"""It is period-{period} out of {nperiod} periods now. Tell me how you think and the value of X you would like to choose. Please answer in json format with keys 'thought' and 'X'. For example, {{"thought": "xxx.","X": x }}. The 'X' should be in 1,2,3,4,5,6,7.
        """.strip()
    else:
        display_table=get_display_table(record)
        message=f"""The history of decisions and your earnings is listed as follows delimited by triple backticks.
```
{display_table}
```
It is period-{period} out of {nperiod} periods now. Tell me how you think and the value of X you would like to choose. Please answer in json format with keys 'thought' and 'X'. For example, {{"thought": "xxx.","X": x }}. The 'X' should be in 1,2,3,4,5,6,7.
    """.strip()
#         message=f"""The history of decisions and your earnings is listed as follows delimited by triple backticks.
# ```
# {display_table}
# ```
# It is period-{period} out of {nperiod} periods now. Tell me how you think and the value of X you would like to choose. Your choice of X should be in 1,2,3,4,5,6,7.
#     """.strip()
    res=bot.chat_wo_update(message, model=model)
    time.sleep(1)
    if display_result:
        print('------get action------')
        print(message)
        print(res)
    out=parse_response(res)
    try:
        X=int(out['X'])
        reason=out['thought']
    except:
        print('key error! messages:',message)
        print('response:',response)
        prompt=summarize_action.replace('<out>',response)
        out=gpt_completion(prompt)
        out=json.loads(out)
        X=int(out['X'])
        reason=out['thought']
        
    try:
        X in [1,2,3,4,5,6,7]
    except:
        print('Invalid action:',out)
        assert 0
    return X,reason

In [22]:
nsess=10

expname='llama13b'


max_try=50
display_result=0

for sess in range(1,1+nsess):
    print(f'==============sess{sess}==============')
    players=[]
    for _ in tqdm(range(N)):
        bot=ChatApp(system_message=system_message)
        players.append(bot)
    records=[]
    cols=['Period','Your choice X','Smallest X','Earnings','Reason of choice']
    for _ in range(N):    
        df = pd.DataFrame(columns=cols)
        records.append(df)
        
    starttime=time.time()
    for period in range(1,nperiod+1):
        print('-----------------------')
        print(f'period-{period}')

        # if this block done, continue
        recordfile=os.path.join(filepath,f'records_ME_{expname}_{N}_{nperiod}_sess{sess}_period{period}.pkl')
        playerfile=os.path.join(filepath,f'players_ME_{expname}_{N}_{nperiod}_sess{sess}_period{period}.pkl')
        if os.path.exists(recordfile) and os.path.exists(playerfile):
            with open(recordfile, 'rb') as f:
                records = pickle.load(f)
            with open(playerfile, 'rb') as f:
                players = pickle.load(f)
            print('existed!')
            continue

        for _ in range(max_try):
            try:
                if period>1:
                    prev_period=period-1
                    recordfile=os.path.join(filepath,f'records_ME_{expname}_{N}_{nperiod}_sess{sess}_period{prev_period}.pkl')
                    playerfile=os.path.join(filepath,f'players_ME_{expname}_{N}_{nperiod}_sess{sess}_period{prev_period}.pkl')
                    with open(recordfile, 'rb') as f:
                        records = pickle.load(f)
                    with open(playerfile, 'rb') as f:
                        players = pickle.load(f)
                # play
                tmp_records=[]
                for i in tqdm(range(N)):
                    bot=players[i]
                    record=records[i]
                    X,reason=get_action(bot,record,model=model,display=display_result)
                    tmp_records.append([period, X, 0.0, 0.0, reason])
                    # cols=['Period','Your choice X','Smallest X','Earnings','Reason of choice']

                # calculate smallest X
                smallest=min([x[1] for x in tmp_records])

                # calculate earnings for all
                for r in tmp_records:
                    earning=get_earnings(r[1],smallest)
                    r[2]=smallest
                    r[3]=earning

                # update records for all
                for i in range(N):
                    record=records[i]
                    record.loc[len(record.index)]=tmp_records[i]

                print('time:',time.time()-starttime)

                # save file
                recordfile=os.path.join(filepath,f'records_ME_{expname}_{N}_{nperiod}_sess{sess}_period{period}.pkl')
                playerfile=os.path.join(filepath,f'players_ME_{expname}_{N}_{nperiod}_sess{sess}_period{period}.pkl')
                with open(recordfile, 'wb') as f:
                    pickle.dump(records, f)
                with open(playerfile, 'wb') as f:
                    pickle.dump(players, f)
                break
            except Exception as e:
                print(e)
                print('error!sleep 30s...')
                time.sleep(30)      

==============sess1==============


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 168521.14it/s]


-----------------------
period-1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:16<00:00,  1.87s/it]


time: 16.792436361312866
-----------------------
period-2


...

time: 106.37627530097961
-----------------------
period-7


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:15<00:00,  1.76s/it]


time: 122.27658534049988
-----------------------
period-8


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:15<00:00,  1.73s/it]

time: 137.86843609809875


In [23]:
# choices=np.zeros((N,nperiod))

nsess=10
period=nperiod

result=[]
for sess in range(1,1+nsess):
    recordfile=os.path.join(filepath,f'records_ME_{expname}_{N}_{nperiod}_sess{sess}_period{period}.pkl')
    with open(recordfile, 'rb') as f:
        records=pickle.load(f)
    for record in records:
        tmp=record['Earnings'].tolist()
        result.extend(tmp)
eff=np.mean((np.array(result)-1)/(13-1))
print('efficiency:',eff)

efficiency: 0.8333333333333334
